# Project Setup & Folder Creation

In [1]:
# EcoSight: Gemini Vision Multi-Agent Waste Sorting
import os
print("🚀 Setting up EcoSight project structure...")

# Create main project directory and all subfolders
!mkdir -p EcoSight/tools EcoSight/agents EcoSight/memory EcoSight/orchestration EcoSight/utils EcoSight/data

# Verify folder structure
print("📁 Project structure created:")
!find EcoSight -type d | sort
print("✅ EcoSight folder structure ready!")

🚀 Setting up EcoSight project structure...
📁 Project structure created:
EcoSight
EcoSight/agents
EcoSight/data
EcoSight/memory
EcoSight/orchestration
EcoSight/tools
EcoSight/utils
✅ EcoSight folder structure ready!


# TrashNet Dataset Setup

In [2]:
# TrashNet Dataset Setup for Kaggle
print("📥 Setting up TrashNet Dataset...")

# The TrashNet dataset on Kaggle is:

dataset_paths = [
    "/kaggle/input/trashnet/data/dataset",  
    "/kaggle/input/trashnet/data",
    "/kaggle/input/trashnet/dataset",
    "/kaggle/input/trashnet",
    "/kaggle/input/garbage-classification/Garbage classification/Garbage classification", 
]

found_path = None
for path in dataset_paths:
    if os.path.exists(path):
        found_path = path
        print(f"✅ Dataset found at: {path}")
        break

if found_path:
    # Copy to our project structure
    !mkdir -p EcoSight/data/trashnet
    !cp -r {found_path}/* EcoSight/data/trashnet/ 2>/dev/null || echo "Using direct access"
    
    # Verify the copy worked
    if os.path.exists("EcoSight/data/trashnet"):
        print("📁 Dataset successfully copied to project structure")
    else:
        print("📁 Using dataset from original location")
else:
    print("❌ TrashNet dataset not found in expected locations")
    print("""
📝 HOW TO ADD THE CORRECT TRASHNET DATASET:

1. Click '+ Add Data' button in Kaggle notebook
2. Search for: "trashnet"
3. Select the dataset: "trashnet" by yangyang111
4. Click 'Add' to attach it to your notebook
5. Wait for dataset to load (green checkmark)
6. Restart session: Session → Restart Session  
7. Re-run all cells

The dataset should then be available at:
/kaggle/input/trashnet/data/dataset/

📊 Original TrashNet Dataset Info:
- 2,527 images across 6 categories
- Categories: cardboard, glass, metal, paper, plastic, trash
- Image sizes: 512x384 pixels
- Research-quality dataset from Stanford
""")

# Verify what we can access
print("\n🔍 Checking dataset structure...")
if found_path:
    !find {found_path} -type d 2>/dev/null | head -15
else:
    # Check our project structure
    !find EcoSight/data/trashnet -type d 2>/dev/null | head -10 || echo "No dataset found"

print("\n✅ TrashNet setup complete!")

📥 Setting up TrashNet Dataset...
✅ Dataset found at: /kaggle/input/trashnet
📁 Dataset successfully copied to project structure

🔍 Checking dataset structure...
/kaggle/input/trashnet
/kaggle/input/trashnet/trashnet
/kaggle/input/trashnet/trashnet/val
/kaggle/input/trashnet/trashnet/val/metal
/kaggle/input/trashnet/trashnet/val/glass
/kaggle/input/trashnet/trashnet/val/paper
/kaggle/input/trashnet/trashnet/val/trash
/kaggle/input/trashnet/trashnet/val/cardboard
/kaggle/input/trashnet/trashnet/val/plastic
/kaggle/input/trashnet/trashnet/test
/kaggle/input/trashnet/trashnet/test/metal
/kaggle/input/trashnet/trashnet/test/glass
/kaggle/input/trashnet/trashnet/test/paper
/kaggle/input/trashnet/trashnet/test/trash
/kaggle/input/trashnet/trashnet/test/cardboard

✅ TrashNet setup complete!


# Import Path Configuration

In [3]:
# Configure Python path to include our project
import sys
import os

# Add EcoSight to Python path
sys.path.append('/kaggle/working/EcoSight')

print("✅ Python path configured for EcoSight/ project structure")
print(f"Project root: /kaggle/working/EcoSight")

✅ Python path configured for EcoSight/ project structure
Project root: /kaggle/working/EcoSight


# Gemini API Setup

In [4]:
# =============================================================================
# 🔑 SECURE GEMINI API CONFIGURATION (Using Kaggle Secrets)
# =============================================================================

import os
from kaggle_secrets import UserSecretsClient

print("🔑 Configuring Gemini AI Integration Securely...")

try:
    # Get Gemini API key from Kaggle Secrets
    GEMINI_API_KEY = UserSecretsClient().get_secret("GEMINI_API_KEY")
    
    if GEMINI_API_KEY and GEMINI_API_KEY.startswith("AIza"):
        os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
        print("✅ Gemini API key loaded securely from Kaggle Secrets!")
        print("🚀 Real object detection will be used")
        print(f"🔐 Key preview: {GEMINI_API_KEY[:10]}...")
    else:
        print("❌ Invalid Gemini API key format in Kaggle Secrets")
        print("💡 Please ensure your key starts with 'AIza'")
        os.environ["GEMINI_API_KEY"] = "MOCK_KEY_NO_VALID_KEY"
        print("🎭 Using mock mode - no valid Gemini API key detected")
        
except Exception as e:
    print(f"🔑 Kaggle Secrets Error: {e}")
    print("""
📝 HOW TO ADD YOUR GEMINI API KEY SECURELY:

1. Click the 'Settings' tab in your Kaggle notebook
2. Scroll down to 'Secrets' section
3. Click 'Add new secret'
4. Set:
   - Name: GEMINI_API_KEY
   - Value: Your actual Gemini API key (starts with AIza...)
5. Click 'Save'
6. Restart your notebook session
7. Re-run this cell

🔒 Security Benefits:
   - API key never appears in your code
   - Key is encrypted and secure
   - Easy to rotate/update without code changes
    """)
    os.environ["GEMINI_API_KEY"] = "MOCK_KEY_SECRETS_ERROR"
    print("🎭 Using mock mode until valid API key is configured")

🔑 Configuring Gemini AI Integration Securely...
✅ Gemini API key loaded securely from Kaggle Secrets!
🚀 Real object detection will be used
🔐 Key preview: AIzaSyBmfe...



# Agent Base Module

**Purpose**: Foundation for the multi-agent system with session management and observability.

**Contains**:

* AgentContext class for workflow state management
* Logger and metrics functions for all agents
* Essential infrastructure for agent communication

In [5]:
%%writefile EcoSight/agents/agent_base.py
# Simplified agent foundation without complex inheritance
import logging
from typing import Any, Dict, Optional
from dataclasses import dataclass

@dataclass
class AgentContext:
    session_id: str
    user_id: Optional[str] = None
    location: Optional[str] = None
    metadata: Dict[str, Any] = None
    
    def __post_init__(self):
        if self.metadata is None:
            self.metadata = {}

def create_agent_logger(name: str):
    """Create a logger for any agent"""
    logger = logging.getLogger(f"agent.{name}")
    return logger

def log_metric(logger, metric_name: str, value: float, tags: Dict[str, str] = None):
    """Log observability metrics for any agent"""
    tags = tags or {}
    tags['agent'] = logger.name
    logger.info(f"METRIC:{metric_name}={value} {tags}")

Writing EcoSight/agents/agent_base.py



# Vision Provider Module

**Purpose**: AI-powered image analysis for waste detection using Gemini Vision API.

**Contains**:

* GeminiVisionProvider for real AI analysis with automatic model discovery
* MockVisionProvider for testing without API key
* Smart filtering and duplicate removal for accurate results

In [6]:
%%writefile EcoSight/tools/vision_provider.py
import random
from abc import ABC, abstractmethod
from typing import Dict, Any
import base64
import os
import asyncio

class VisionProvider(ABC):
    @abstractmethod
    async def analyze_image(self, image_b64: str) -> Dict[str, Any]:
        pass

class MockVisionProvider(VisionProvider):
    """Mock vision provider for testing - uses TrashNet categories"""
    
    async def analyze_image(self, image_b64: str) -> Dict[str, Any]:
        await asyncio.sleep(0.1)
        
        # Mock detection based on TrashNet categories
        trashnet_categories = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
        detected_category = random.choice(trashnet_categories)
        
        mock_items = [
            {"name": f"{detected_category} item", "confidence": random.uniform(0.85, 0.98), "bbox": [100, 100, 50, 50]},
        ]
        
        # Sometimes detect multiple items (1-4 total)
        additional_items = random.randint(0, 3)
        for _ in range(additional_items):
            second_category = random.choice([c for c in trashnet_categories if c != detected_category])
            mock_items.append({
                "name": f"{second_category} item", 
                "confidence": random.uniform(0.75, 0.90), 
                "bbox": [random.randint(50, 300), random.randint(50, 200), 40, 40]
            })
        
        return {
            "items": mock_items,
            "analysis_confidence": random.uniform(0.8, 0.95),
            "model_version": "mock-trashnet-v1.0",
            "provider": "mock"
        }

class GeminiVisionProvider(VisionProvider):
    """Real Google Gemini Vision implementation"""
    
    def __init__(self):
        self.api_key = os.getenv("GEMINI_API_KEY")
        if not self.api_key or self.api_key == "MOCK_MODE_NO_VALID_KEY":
            raise ValueError("Gemini API key not found. Please set GEMINI_API_KEY in Kaggle Secrets")
        
        # Initialize Gemini client
        try:
            import google.generativeai as genai
            self.genai = genai
            genai.configure(api_key=self.api_key)
            
            # Get available models and find a working one
            self.model, self.model_name = self._get_working_model()
            print(f"✅ Gemini Vision initialized with model: {self.model_name}")
            
        except ImportError:
            raise ImportError("Google GenAI library not installed. Run: pip install google-generativeai")
    
    def _get_working_model(self):
        """Get a working Gemini model"""
        try:
            # Try the most common model names for the new library
            model_candidates = [
                "gemini-1.5-flash",
                "gemini-1.5-pro",
                "gemini-pro",
                "gemini-pro-vision"
            ]
            
            for model_name in model_candidates:
                try:
                    model = self.genai.GenerativeModel(model_name)
                    # Test with a simple prompt
                    response = model.generate_content("Test")
                    print(f"✅ Model {model_name} is working")
                    return model, model_name
                except Exception as e:
                    print(f"❌ Model {model_name} failed: {str(e)[:100]}...")
                    continue
            
            # If no specific model works, try to list available models
            print("🔍 Listing available models...")
            available_models = self.genai.list_models()
            for model in available_models:
                model_name = model.name
                if 'generateContent' in model.supported_generation_methods:
                    print(f"📋 Available: {model_name}")
                    try:
                        # Extract just the model name without "models/" prefix
                        simple_name = model_name.replace("models/", "")
                        test_model = self.genai.GenerativeModel(simple_name)
                        response = test_model.generate_content("Test")
                        print(f"✅ Using model: {model_name}")
                        return test_model, model_name
                    except Exception as e:
                        # Try with full model name including "models/" prefix
                        try:
                            test_model = self.genai.GenerativeModel(model_name)
                            response = test_model.generate_content("Test")
                            print(f"✅ Using model with full name: {model_name}")
                            return test_model, model_name
                        except:
                            continue
            
            raise Exception("No working Gemini model found")
            
        except Exception as e:
            print(f"❌ Model discovery failed: {e}")
            raise
    
    async def analyze_image(self, image_b64: str) -> Dict[str, Any]:
        """Analyze image using real Gemini Vision API"""
        try:
            print(f"🔮 Calling real Gemini Vision API with model: {self.model_name}...")
            
            # Decode base64 image
            image_data = base64.b64decode(image_b64)
            
            # Enhanced prompt for better waste detection
            prompt = """
            Analyze this image and identify ALL waste items visible. Be thorough and comprehensive.
            
            For EACH distinct waste item found, provide:
            1. Specific type (e.g., "plastic water bottle", "glass jar", "metal can", "cardboard box", "paper cup")
            2. Material composition 
            3. Approximate size/condition
            4. Whether it appears to be recyclable, compostable, or landfill
            
            IMPORTANT: 
            - List EACH item separately even if they are the same material
            - Be specific about the item type rather than generic
            - Include ALL items you can identify
            - Focus on household waste items commonly found in recycling/trash
            
            Return a clear list of all detected waste items.
            """
            
            # Call Gemini API
            response = await asyncio.get_event_loop().run_in_executor(
                None,
                lambda: self.model.generate_content([prompt, {"mime_type": "image/jpeg", "data": image_data}])
            )
            
            # Parse the response
            detected_items = self._parse_gemini_response(response.text)
            
            return {
                "items": detected_items,
                "analysis_confidence": 0.92,
                "model_version": self.model_name,
                "provider": "gemini",
                "raw_response": response.text[:300] + "..." if len(response.text) > 300 else response.text,
                "total_detected": len(detected_items)
            }
            
        except Exception as e:
            print(f"❌ Gemini Vision API error: {e}")
            # Fallback to mock provider
            print("🔄 Falling back to mock provider...")
            mock_provider = MockVisionProvider()
            return await mock_provider.analyze_image(image_b64)
    
    def _parse_gemini_response(self, response_text: str) -> list:
        """Parse Gemini response with enhanced filtering and better item naming"""
        items = []
        
        # Enhanced waste keywords with more specific item types
        waste_categories = {
            'plastic': {
                'keywords': ['plastic', 'bottle', 'container', 'bag', 'wrapper', 'packaging', 'cup', 'utensil', 'straw'],
                'items': ['water bottle', 'soda bottle', 'food container', 'shopping bag', 'chip bag', 'yogurt cup']
            },
            'paper': {
                'keywords': ['paper', 'cardboard', 'newspaper', 'magazine', 'box', 'carton', 'tissue', 'napkin'],
                'items': ['cardboard box', 'newspaper', 'magazine', 'paper bag', 'egg carton']
            },
            'glass': {
                'keywords': ['glass', 'bottle', 'jar', 'container', 'window'],
                'items': ['glass bottle', 'jar', 'wine bottle', 'beer bottle']
            },
            'metal': {
                'keywords': ['metal', 'can', 'aluminum', 'tin', 'container', 'foil'],
                'items': ['soda can', 'food can', 'aluminum foil', 'metal container']
            },
            'cardboard': {
                'keywords': ['cardboard', 'box', 'packaging', 'carton'],
                'items': ['cardboard box', 'shipping box', 'cereal box', 'pizza box']
            },
            'organic': {
                'keywords': ['food', 'organic', 'fruit', 'vegetable', 'compost', 'banana', 'apple', 'peel'],
                'items': ['fruit peel', 'vegetable scraps', 'food waste', 'coffee grounds']
            },
            'trash': {
                'keywords': ['trash', 'garbage', 'waste', 'debris', 'litter', 'general waste'],
                'items': ['general waste', 'mixed trash', 'non-recyclable item']
            }
        }
        
        lines = response_text.split('\n')
        current_item = {}
        
        for line in lines:
            line = line.strip()
            if not line or len(line) < 10:  # Skip very short lines
                continue
                
            # Skip lines that are likely headers or section titles
            if any(skip in line.lower() for skip in ['analysis:', 'items:', 'detected:', 'list:', 'waste:']):
                continue
                
            # Skip numbered items if they're just numbers
            if line.replace('.', '').strip().isdigit():
                continue
                
            # Remove common prefixes like "1.", "-", "•", "*"
            clean_line = self._clean_line(line)
            if not clean_line:
                continue
                
            # Determine category and create specific item name
            detected_category, item_name, confidence = self._categorize_item(clean_line, waste_categories)
            
            # Create unique bounding box
            bbox = [
                random.randint(50, 400),
                random.randint(50, 300),
                random.randint(40, 120),
                random.randint(40, 120)
            ]
            
            items.append({
                "name": item_name,
                "confidence": confidence,
                "bbox": bbox,
                "description": clean_line[:100],
                "category_hint": detected_category
            })
        
        # Enhanced filtering
        if not items:
            items.append({
                "name": "unidentified waste item",
                "confidence": 0.5,
                "bbox": [100, 100, 50, 50],
                "description": "Waste item detected but not specifically identified",
                "category_hint": "trash"
            })
        
        # Apply smart filtering
        filtered_items = self._apply_smart_filtering(items)
        
        print(f"📊 Detected {len(items)} raw items → {len(filtered_items)} after enhanced filtering")
        
        return filtered_items
    
    def _clean_line(self, line: str) -> str:
        """Clean and normalize a line of text"""
        # Remove common prefixes
        prefixes = ['- ', '• ', '* ', '1. ', '2. ', '3. ', '4. ', '5. ', '6. ', '7. ', '8. ', '9. ', '10. ']
        for prefix in prefixes:
            if line.startswith(prefix):
                line = line[len(prefix):]
                break
                
        # Remove content in parentheses and brackets
        import re
        line = re.sub(r'\([^)]*\)', '', line)  # Remove (content)
        line = re.sub(r'\[[^\]]*\]', '', line)  # Remove [content]
        
        return line.strip()
    
    def _categorize_item(self, text: str, waste_categories: dict) -> tuple:
        """Categorize item and generate specific name"""
        text_lower = text.lower()
        detected_category = 'trash'
        confidence = 0.7
        item_name = "waste item"
        
        # Find the best matching category
        best_match_score = 0
        best_category = 'trash'
        
        for category, data in waste_categories.items():
            keyword_matches = [kw for kw in data['keywords'] if kw in text_lower]
            match_score = len(keyword_matches)
            
            if match_score > best_match_score:
                best_match_score = match_score
                best_category = category
        
        detected_category = best_category
        
        # Boost confidence based on match quality
        confidence = min(0.95, 0.7 + (best_match_score * 0.08))
        
        # Generate specific item name
        if detected_category != 'trash':
            # Try to find a specific item type from our list
            category_items = waste_categories[detected_category]['items']
            for specific_item in category_items:
                if any(word in text_lower for word in specific_item.split()):
                    item_name = specific_item
                    confidence = min(0.98, confidence + 0.1)  # Boost for specific match
                    break
            else:
                # Use generic category name
                item_name = f"{detected_category} item"
        else:
            # For trash, try to extract key words
            words = text_lower.split()
            if len(words) > 1:
                item_name = f"{words[0]} {words[1]}" if len(words) >= 2 else words[0]
            else:
                item_name = "general waste"
        
        return detected_category, item_name, round(confidence, 2)
    
    def _apply_smart_filtering(self, items: list) -> list:
        """Apply enhanced filtering to remove duplicates and low-quality items"""
        # 1. Remove very low confidence items
        filtered_items = [item for item in items if item['confidence'] > 0.55]
        
        # 2. Enhanced duplicate removal
        unique_items = self._remove_duplicates_enhanced(filtered_items)
        
        # 3. Sort by confidence (highest first)
        sorted_items = sorted(unique_items, key=lambda x: x['confidence'], reverse=True)
        
        # 4. Adaptive limit based on quality of detections
        high_confidence_count = len([item for item in sorted_items if item['confidence'] > 0.8])
        
        if high_confidence_count >= 8:
            limit = min(20, len(sorted_items))  # Many good detections - show more
        elif high_confidence_count >= 4:
            limit = min(15, len(sorted_items))  # Moderate good detections
        else:
            limit = min(10, len(sorted_items))  # Few good detections - be conservative
        
        return sorted_items[:limit]
    
    def _remove_duplicates_enhanced(self, items: list) -> list:
        """Enhanced duplicate removal with similarity scoring"""
        unique_items = []
        seen_signatures = set()
        
        for item in items:
            # Create a signature based on name and key description words
            name_words = set(item['name'].lower().split())
            desc_words = set(item['description'].lower().split()[:5])  # First 5 words
            
            # Combine for uniqueness check
            signature_parts = list(name_words | desc_words)
            signature_parts.sort()
            signature = '_'.join(signature_parts[:4])  # Use first 4 combined words
            
            if signature not in seen_signatures:
                seen_signatures.add(signature)
                unique_items.append(item)
            else:
                # If duplicate found, keep the one with higher confidence
                existing_idx = None
                for i, existing in enumerate(unique_items):
                    existing_sig_parts = list(set(existing['name'].lower().split()) | set(existing['description'].lower().split()[:5]))
                    existing_sig_parts.sort()
                    existing_sig = '_'.join(existing_sig_parts[:4])
                    
                    if existing_sig == signature and item['confidence'] > existing['confidence']:
                        existing_idx = i
                        break
                
                if existing_idx is not None:
                    unique_items[existing_idx] = item  # Replace with higher confidence item
        
        return unique_items

Writing EcoSight/tools/vision_provider.py


# Waste Database Tool

Waste Database Module

**Purpose**: Waste classification system aligned with TrashNet categories and recycling rules.

**Contains**:
* Classification rules for recyclable/compost/landfill materials
* Location-aware waste processing guidelines
* Support for all TrashNet waste categories

In [7]:
%%writefile EcoSight/tools/waste_db.py
from typing import Dict, Any, Optional

class WasteDB:
    """Tool for waste classification database - aligned with TrashNet categories"""
    
    def __init__(self, db_path: Optional[str] = None):
        self.classification_rules = self._load_default_rules()
    
    def _load_default_rules(self) -> Dict[str, Any]:
        # Classification rules aligned with TrashNet categories
        return {
            "cardboard": {"category": "recyclable", "material": "cardboard", "processing": "flatten"},
            "glass": {"category": "recyclable", "material": "glass", "processing": "separate_colors"},
            "metal": {"category": "recyclable", "material": "metal", "processing": "clean"},
            "paper": {"category": "recyclable", "material": "paper", "processing": "dry"},
            "plastic": {"category": "recyclable", "material": "plastic", "processing": "check_code"},
            "trash": {"category": "landfill", "material": "mixed", "notes": "non-recyclable"},
            # Additional common items
            "bottle": {"category": "recyclable", "material": "plastic", "processing": "rinse"},
            "can": {"category": "recyclable", "material": "metal", "processing": "rinse"},
            "container": {"category": "recyclable", "material": "mixed", "processing": "check_label"},
        }
    
    async def classify_item(self, item_name: str, location: Optional[str] = None) -> Dict[str, Any]:
        item_lower = item_name.lower()
        
        # Match against TrashNet categories and common patterns
        for key, rule in self.classification_rules.items():
            if key in item_lower:
                result = rule.copy()
                # Add location-specific adjustments if needed
                if location and self._has_location_override(location, key):
                    result.update(self._get_location_override(location, key))
                return result
        
        # Default classification for unknown items
        return {"category": "landfill", "material": "unknown", "notes": "unidentified"}
    
    def _has_location_override(self, location: str, item_key: str) -> bool:
        # Simplified location override check
        return False
    
    def _get_location_override(self, location: str, item_key: str) -> Dict[str, Any]:
        # Simplified location override rules
        return {}

Writing EcoSight/tools/waste_db.py


# Location Finder Tool

**Purpose**: Provides location-specific waste disposal instructions and recycling guidelines.

**Contains**:
* City-specific recycling rules (NYC, SF, default)
* Disposal facility information and preparation instructions
* Geographic-aware waste management recommendations

In [8]:
%%writefile EcoSight/tools/location_finder.py
from typing import Dict, Any, List, Optional

class LocationFinder:
    """Tool for location-specific waste disposal rules"""
    
    def __init__(self):
        self.location_rules = self._load_default_rules()
    
    def _load_default_rules(self) -> Dict[str, Any]:
        return {
            "NYC": {
                "recyclable": {
                    "instructions": "Place in blue recycling bin - separate paper and metal/glass/plastic",
                    "facilities": ["Curbside pickup", "DSNY recycling centers"],
                    "preparation": "Rinse containers, flatten boxes, no plastic bags"
                },
                "compost": {
                    "instructions": "Brown bin or designated compost drop-off",
                    "facilities": ["DSNY compost sites", "Farmer's markets"],
                    "preparation": "Use compostable bags, no plastic contamination"
                },
                "landfill": {
                    "instructions": "Black bin for non-recyclable waste",
                    "facilities": ["Curbside pickup"],
                    "preparation": "Bag securely to prevent litter"
                }
            },
            "SF": {
                "recyclable": {
                    "instructions": "Blue bin for mixed recyclables",
                    "facilities": ["Curbside pickup", "Recology centers"],
                    "preparation": "No plastic bags, rinse containers"
                },
                "compost": {
                    "instructions": "Green bin for compostables",
                    "facilities": ["Curbside pickup", "Community compost"],
                    "preparation": "No plastic contamination"
                }
            },
            "default": {
                "recyclable": {
                    "instructions": "Check local recycling guidelines",
                    "facilities": ["Local recycling center"],
                    "preparation": "Rinse and sort by material"
                },
                "compost": {
                    "instructions": "Compost bin or municipal collection",
                    "facilities": ["Local compost facility"],
                    "preparation": "No plastic contamination"
                },
                "landfill": {
                    "instructions": "Regular trash bin",
                    "facilities": ["Curbside pickup", "Landfill site"],
                    "preparation": "Bag securely"
                }
            }
        }
    
    async def get_disposal_instructions(self, category: str, location: Optional[str] = None) -> Dict[str, Any]:
        location_key = location if location and location in self.location_rules else "default"
        
        # Get category rules with fallback
        category_rules = self.location_rules[location_key].get(
            category, 
            self.location_rules["default"].get(category, {})
        )
        
        return {
            "instructions": category_rules.get("instructions", "Dispose according to local regulations"),
            "facilities": category_rules.get("facilities", []),
            "preparation": category_rules.get("preparation", "Follow general guidelines"),
            "location": location_key
        }

Writing EcoSight/tools/location_finder.py


# PDF Generator Tool

**Purpose**: Creates professional, branded PDF reports with comprehensive waste analysis and environmental impact data.

**Contains**:

* Executive summaries with performance metrics
* Visual waste composition breakdowns
* Environmental impact assessments with equivalent comparisons
* AI-powered personalized recommendations

In [9]:
%%writefile EcoSight/tools/pdf_generator.py

from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.graphics.shapes import Drawing, Rect
from reportlab.graphics.charts.barcharts import VerticalBarChart
import io
from datetime import datetime
from typing import Dict, Any
import os

class PDFGenerator:
    """Enhanced PDF generator with professional styling and branding"""
    
    def __init__(self):
        self.styles = getSampleStyleSheet()
        self._setup_custom_styles()
    
    def _setup_custom_styles(self):
        """Setup enhanced custom styles for professional reports"""
        # EcoSight color scheme
        self.primary_color = colors.HexColor('#2E7D32')  # Eco green
        self.secondary_color = colors.HexColor('#4CAF50')  # Light green
        self.accent_color = colors.HexColor('#FF9800')    # Orange
        self.dark_color = colors.HexColor('#1B5E20')      # Dark green
        self.light_color = colors.HexColor('#E8F5E8')     # Light background
        
        # Enhanced title style
        self.styles.add(ParagraphStyle(
            name='EcoTitle',
            parent=self.styles['Title'],
            fontSize=24,
            spaceAfter=20,
            textColor=self.primary_color,
            alignment=1,  # Center
            fontName='Helvetica-Bold'
        ))
        
        # Section header style
        self.styles.add(ParagraphStyle(
            name='EcoHeading1',
            parent=self.styles['Heading1'],
            fontSize=16,
            spaceAfter=12,
            textColor=self.dark_color,
            fontName='Helvetica-Bold',
            leftIndent=10
        ))
        
        # Subsection style
        self.styles.add(ParagraphStyle(
            name='EcoHeading2',
            parent=self.styles['Heading2'],
            fontSize=14,
            spaceAfter=8,
            textColor=self.primary_color,
            fontName='Helvetica-Bold'
        ))
        
        # Body text with better spacing
        self.styles.add(ParagraphStyle(
            name='EcoBody',
            parent=self.styles['BodyText'],
            fontSize=10,
            spaceAfter=6,
            textColor=colors.black,
            fontName='Helvetica'
        ))
        
        # Highlight style for important numbers
        self.styles.add(ParagraphStyle(
            name='EcoHighlight',
            parent=self.styles['BodyText'],
            fontSize=11,
            textColor=self.primary_color,
            fontName='Helvetica-Bold',
            backColor=self.light_color,
            borderPadding=5,
            spaceAfter=8
        ))
        
        # Footer style
        self.styles.add(ParagraphStyle(
            name='EcoFooter',
            parent=self.styles['Italic'],
            fontSize=8,
            textColor=colors.gray,
            alignment=1  # Center
        ))
    
    def _create_header(self, story):
        """Create professional header with branding"""
        # Header with logo placeholder and title
        header_table_data = [
            ['ECO GUARDIAN', 'AI-Powered Waste Analysis']
        ]
        
        header_table = Table(header_table_data, colWidths=[3*inch, 3*inch])
        header_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (0, 0), self.primary_color),
            ('BACKGROUND', (1, 0), (1, 0), self.secondary_color),
            ('TEXTCOLOR', (0, 0), (-1, -1), colors.white),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (0, 0), 16),
            ('FONTSIZE', (1, 0), (1, 0), 12),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('BOX', (0, 0), (-1, -1), 1, colors.white),
            ('LINEBELOW', (0, 0), (-1, -1), 1, colors.white),
        ]))
        
        story.append(header_table)
        story.append(Spacer(1, 0.1*inch))
    
    def _create_cover_page(self, story, session_id):
        """Create a professional cover page"""
        # Title
        story.append(Paragraph("WASTE ANALYSIS REPORT", self.styles['EcoTitle']))
        story.append(Spacer(1, 0.3*inch))
        
        # Session info in a styled box
        info_data = [
            ['Session ID:', session_id],
            ['Generated:', datetime.now().strftime('%Y-%m-%d %H:%M')],
            ['Report Type:', 'Comprehensive Waste Analysis'],
            ['AI Model:', 'Gemini Vision + Multi-Agent System']
        ]
        
        info_table = Table(info_data, colWidths=[1.5*inch, 4*inch])
        info_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (0, -1), self.light_color),
            ('BACKGROUND', (1, 0), (1, -1), colors.white),
            ('TEXTCOLOR', (0, 0), (-1, -1), colors.black),
            ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
            ('FONTNAME', (1, 0), (1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 0), (-1, -1), 10),
            ('GRID', (0, 0), (-1, -1), 1, self.primary_color),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('PADDING', (0, 0), (-1, -1), 6),
        ]))
        
        story.append(info_table)
        story.append(Spacer(1, 0.5*inch))
        
        # Mission statement
        mission_text = """
        <b>Our Mission:</b> To leverage artificial intelligence for smarter waste management, 
        promoting recycling efficiency and environmental sustainability through advanced 
        multi-agent systems and computer vision technology.
        """
        story.append(Paragraph(mission_text, self.styles['EcoBody']))
        
        # Page break for next section
        story.append(Spacer(1, 0.5*inch))
        story.append(Paragraph("--- Report Contents ---", self.styles['EcoFooter']))
    
    def _create_executive_summary(self, story, report_data):
        """Create enhanced executive summary with visual elements"""
        story.append(Paragraph("Executive Summary", self.styles['EcoHeading1']))
        story.append(Spacer(1, 0.1*inch))
        
        summary = report_data.get('summary', {})
        environmental = report_data.get('environmental_impact', {})
        
        # Key metrics in a professional table
        metrics_data = [
            ['METRIC', 'VALUE', 'IMPACT'],
            ['Total Items Analyzed', str(summary.get('total_items', 0)), 'Analysis Scope'],
            ['Recycling Rate', f"{summary.get('recyclable_percent', 0):.1f}%", 'Efficiency Score'],
            ['CO₂ Reduction', f"{environmental.get('co2_saved_kg', 0):.1f} kg", 'Environmental Impact'],
            ['Water Saved', f"{environmental.get('water_saved_liters', 0):.0f} L", 'Resource Conservation'],
            ['Energy Saved', f"{environmental.get('energy_saved_kwh', 0):.1f} kWh", 'Energy Efficiency']
        ]
        
        metrics_table = Table(metrics_data, colWidths=[1.8*inch, 1.5*inch, 2.2*inch])
        metrics_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), self.primary_color),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 0), (-1, -1), 9),
            ('GRID', (0, 0), (-1, -1), 1, colors.lightgrey),
            ('BACKGROUND', (0, 1), (-1, -1), colors.white),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, self.light_color]),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('PADDING', (0, 0), (-1, -1), 6),
        ]))
        
        story.append(metrics_table)
        story.append(Spacer(1, 0.2*inch))
        
        # Performance indicator
        recyclable_rate = summary.get('recyclable_percent', 0)
        if recyclable_rate >= 70:
            performance = "Excellent"
            color = self.primary_color
        elif recyclable_rate >= 50:
            performance = "Good"
            color = self.accent_color
        else:
            performance = "Needs Improvement"
            color = colors.red
        
        performance_text = f"""
        <b>Performance Rating:</b> <font color="{color.toHex()}">{performance}</font><br/>
        <b>Overall Assessment:</b> Your waste stream shows a recycling efficiency of {recyclable_rate:.1f}%. 
        This analysis provides actionable insights to optimize your waste management practices.
        """
        story.append(Paragraph(performance_text, self.styles['EcoBody']))
        story.append(Spacer(1, 0.2*inch))
    
    def _create_waste_breakdown(self, story, report_data):
        """Create visual waste breakdown section"""
        story.append(Paragraph("Waste Composition Analysis", self.styles['EcoHeading1']))
        story.append(Spacer(1, 0.1*inch))
        
        summary = report_data.get('summary', {})
        
        # Waste distribution table
        distribution_data = [
            ['CATEGORY', 'PERCENTAGE', 'ITEMS'],
            ['Recyclable', f"{summary.get('recyclable_percent', 0):.1f}%", 'Plastic, Metal, Glass, Paper'],
            ['Compostable', f"{summary.get('compost_percent', 0):.1f}%", 'Food Waste, Organic Materials'],
            ['Landfill', f"{summary.get('landfill_percent', 0):.1f}%", 'Non-Recyclable Items'],
            ['Hazardous', f"{summary.get('hazardous_percent', 0):.1f}%", 'Special Handling Required']
        ]
        
        dist_table = Table(distribution_data, colWidths=[1.5*inch, 1.2*inch, 3*inch])
        dist_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), self.dark_color),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
            ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 0), (-1, -1), 9),
            ('GRID', (0, 0), (-1, -1), 1, colors.lightgrey),
            ('BACKGROUND', (0, 1), (0, 1), self.secondary_color),
            ('BACKGROUND', (0, 2), (0, 2), colors.orange),
            ('BACKGROUND', (0, 3), (0, 3), colors.lightgrey),
            ('BACKGROUND', (0, 4), (0, 4), colors.red),
            ('TEXTCOLOR', (0, 1), (0, 4), colors.white),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('PADDING', (0, 0), (-1, -1), 6),
        ]))
        
        story.append(dist_table)
        story.append(Spacer(1, 0.2*inch))
    
    def _create_detailed_analysis(self, story, report_data):
        """Create enhanced detailed analysis section"""
        story.append(Paragraph("Detailed Item Analysis", self.styles['EcoHeading1']))
        story.append(Spacer(1, 0.1*inch))
        
        breakdown = report_data.get('detailed_breakdown', [])
        
        if breakdown:
            # Create professional table for item analysis
            table_data = [['ITEM', 'CATEGORY', 'DISPOSAL GUIDANCE', 'CONFIDENCE']]
            
            for item in breakdown:
                table_data.append([
                    item.get('item', 'Unknown'),
                    item.get('category', 'Unknown').title(),
                    item.get('disposal_instructions', {}).get('instructions', 'N/A'),
                    f"{item.get('confidence', 0)*100:.1f}%"
                ])
            
            analysis_table = Table(table_data, colWidths=[1.2*inch, 1*inch, 2.2*inch, 0.8*inch])
            analysis_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), self.primary_color),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
                ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                ('ALIGN', (3, 1), (3, -1), 'CENTER'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
                ('FONTSIZE', (0, 0), (-1, -1), 8),
                ('GRID', (0, 0), (-1, -1), 1, colors.lightgrey),
                ('BACKGROUND', (0, 1), (-1, -1), colors.white),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, self.light_color]),
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
                ('PADDING', (0, 0), (-1, -1), 4),
            ]))
            
            story.append(analysis_table)
        else:
            story.append(Paragraph("No detailed item data available for this analysis.", self.styles['EcoBody']))
        
        story.append(Spacer(1, 0.2*inch))
    
    def _create_environmental_impact(self, story, report_data):
        """Create enhanced environmental impact section"""
        story.append(Paragraph("Environmental Impact Assessment", self.styles['EcoHeading1']))
        story.append(Spacer(1, 0.1*inch))
        
        impact = report_data.get('environmental_impact', {})
        
        impact_data = [
            ['ENVIRONMENTAL METRIC', 'AMOUNT SAVED', 'EQUIVALENT TO'],
            ['CO₂ Emissions', f"{impact.get('co2_saved_kg', 0):.1f} kg", 'Driving 5 miles in a car'],
            ['Water Usage', f"{impact.get('water_saved_liters', 0):.0f} liters", '100 showers'],
            ['Energy Consumption', f"{impact.get('energy_saved_kwh', 0):.1f} kWh", 'Powering a home for 1 day']
        ]
        
        impact_table = Table(impact_data, colWidths=[2*inch, 1.5*inch, 2*inch])
        impact_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), self.accent_color),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 0), (-1, -1), 9),
            ('GRID', (0, 0), (-1, -1), 1, colors.lightgrey),
            ('BACKGROUND', (0, 1), (-1, -1), colors.white),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('PADDING', (0, 0), (-1, -1), 6),
        ]))
        
        story.append(impact_table)
        story.append(Spacer(1, 0.2*inch))
        
        # Impact summary
        total_impact = sum([impact.get('co2_saved_kg', 0), impact.get('water_saved_liters', 0)/100, impact.get('energy_saved_kwh', 0)])
        impact_text = f"""
        <b>Overall Environmental Impact Score:</b> {total_impact:.1f} points<br/>
        Your recycling efforts are making a measurable difference in reducing environmental footprint 
        and promoting sustainable waste management practices.
        """
        story.append(Paragraph(impact_text, self.styles['EcoHighlight']))
        story.append(Spacer(1, 0.2*inch))
    
    def _create_recommendations(self, story, report_data):
        """Create enhanced recommendations section"""
        story.append(Paragraph("AI-Powered Recommendations", self.styles['EcoHeading1']))
        story.append(Spacer(1, 0.1*inch))
        
        tips = report_data.get('personalized_tips', [])
        
        if tips:
            for i, tip in enumerate(tips, 1):
                tip_text = f"<b>Recommendation {i}:</b> {tip}"
                story.append(Paragraph(tip_text, self.styles['EcoBody']))
                story.append(Spacer(1, 0.1*inch))
        else:
            story.append(Paragraph("No specific recommendations available for this analysis.", self.styles['EcoBody']))
        
        story.append(Spacer(1, 0.2*inch))
        
        # General best practices
        best_practices = [
            "✅ Always rinse recyclable containers before disposal",
            "✅ Separate materials according to local guidelines", 
            "✅ Reduce single-use plastics when possible",
            "✅ Compost food waste to minimize landfill impact",
            "✅ Stay informed about local recycling program updates"
        ]
        
        story.append(Paragraph("<b>General Best Practices:</b>", self.styles['EcoHeading2']))
        for practice in best_practices:
            story.append(Paragraph(practice, self.styles['EcoBody']))
            story.append(Spacer(1, 0.05*inch))
    
    def _create_footer(self, story):
        """Create professional footer"""
        story.append(Spacer(1, 0.3*inch))
        
        footer_text = """
        <b>EcoSight AI System</b><br/>
        Multi-Agent Waste Detection & Recycling Optimization Platform<br/>
        Generated with Advanced Computer Vision and AI Analysis<br/>
        <i>Driving sustainability through intelligent waste management</i>
        """
        story.append(Paragraph(footer_text, self.styles['EcoFooter']))
    
    async def generate_EcoSight_report(self, report_data: Dict[str, Any], session_id: str) -> bytes:
        """Generate enhanced professional PDF report"""
        
        buffer = io.BytesIO()
        doc = SimpleDocTemplate(buffer, pagesize=A4, topMargin=0.5*inch, bottomMargin=0.5*inch)
        story = []
        
        # Build enhanced PDF content
        self._create_header(story)
        self._create_cover_page(story, session_id)
        self._create_executive_summary(story, report_data)
        self._create_waste_breakdown(story, report_data)
        self._create_detailed_analysis(story, report_data) 
        self._create_environmental_impact(story, report_data)
        self._create_recommendations(story, report_data)
        self._create_footer(story)
        
        # Build PDF
        doc.build(story)
        pdf_bytes = buffer.getvalue()
        buffer.close()
        
        return pdf_bytes

Writing EcoSight/tools/pdf_generator.py


# PDF Downloader Utility

**Purpose**: Provides interactive PDF viewing, downloading, and preview options with professional UI.

**Contains**:

* Interactive dashboard with toggle preview
* Multiple download formats and file management
* Embedded PDF viewer with toolbar controls
* File size tracking and download options



In [10]:
%%writefile EcoSight/utils/pdf_downloader.py
import base64
from IPython.display import HTML, display, IFrame
from typing import Optional
import tempfile
import os

class PDFDownloader:
    """Enhanced utility for PDF handling with preview and download functionality"""
    
    @staticmethod
    def create_download_link(pdf_bytes: bytes, filename: str, link_text: str = "Download PDF Report") -> HTML:
        """Create a downloadable link for PDF in Kaggle notebook"""
        try:
            b64 = base64.b64encode(pdf_bytes).decode()
            download_html = f'''
            <div style="padding: 15px; border: 2px solid #4CAF50; border-radius: 8px; background-color: #f9f9f9; margin: 10px 0;">
                <div style="display: flex; align-items: center; justify-content: space-between;">
                    <div>
                        <strong style="color: #2E7D32;">📄 {filename}</strong><br>
                        <span style="color: #666; font-size: 12px;">Size: {len(pdf_bytes)/1024:.1f} KB</span>
                    </div>
                    <a href="data:application/pdf;base64,{b64}" download="{filename}" 
                       style="background-color: #4CAF50; color: white; padding: 10px 20px; 
                              text-decoration: none; border-radius: 5px; font-weight: bold;
                              transition: background-color 0.3s;">
                       {link_text}
                    </a>
                </div>
            </div>
            '''
            return HTML(download_html)
        except Exception as e:
            error_html = f'''
            <div style="color: red; padding: 10px; border: 1px solid red; border-radius: 5px;">
                ❌ Error creating download link: {str(e)}
            </div>
            '''
            return HTML(error_html)
    
    @staticmethod
    def create_pdf_preview(pdf_bytes: bytes, filename: str, width: str = "100%", height: str = "600px") -> HTML:
        """Create an embedded PDF preview with download options"""
        try:
            b64 = base64.b64encode(pdf_bytes).decode()
            
            preview_html = f'''
            <div style="border: 2px solid #4CAF50; border-radius: 10px; padding: 15px; margin: 15px 0; background: white;">
                <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 15px; padding-bottom: 10px; border-bottom: 1px solid #eee;">
                    <h3 style="color: #2E7D32; margin: 0;">🔍 PDF Preview: {filename}</h3>
                    <div style="display: flex; gap: 10px;">
                        <a href="data:application/pdf;base64,{b64}" download="{filename}" 
                           style="background-color: #4CAF50; color: white; padding: 8px 16px; 
                                  text-decoration: none; border-radius: 4px; font-size: 14px;">
                           📥 Download
                        </a>
                    </div>
                </div>
                
                <div style="background: #f5f5f5; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
                    <strong>File Info:</strong> {len(pdf_bytes)/1024:.1f} KB • Generated by EcoSight AI
                </div>
                
                <iframe src="data:application/pdf;base64,{b64}" 
                        width="{width}" 
                        height="{height}" 
                        style="border: 1px solid #ddd; border-radius: 5px;">
                </iframe>
                
                <div style="margin-top: 10px; text-align: center; color: #666; font-size: 12px;">
                    💡 Scroll to navigate • Use download button above to save
                </div>
            </div>
            '''
            return HTML(preview_html)
        except Exception as e:
            error_html = f'''
            <div style="color: red; padding: 15px; border: 1px solid red; border-radius: 5px; margin: 10px 0;">
                ❌ Error creating PDF preview: {str(e)}
            </div>
            '''
            return HTML(error_html)
    
    @staticmethod
    def create_interactive_pdf_dashboard(pdf_bytes: bytes, filename: str) -> HTML:
        """Create an interactive PDF dashboard with preview and multiple download options"""
        try:
            b64 = base64.b64encode(pdf_bytes).decode()
            file_size_kb = len(pdf_bytes) / 1024
            
            dashboard_html = f'''
            <div style="border: 2px solid #2E7D32; border-radius: 12px; padding: 20px; margin: 20px 0; background: linear-gradient(135deg, #f8fff8, #e8f5e8);">
                <!-- Header -->
                <div style="text-align: center; margin-bottom: 20px;">
                    <h2 style="color: #2E7D32; margin: 0 0 5px 0;">🌿 EcoSight Report Ready</h2>
                    <p style="color: #666; margin: 0;">AI-Powered Waste Analysis Complete</p>
                </div>
                
                <!-- File Info Card -->
                <div style="background: white; padding: 15px; border-radius: 8px; margin-bottom: 20px; border-left: 4px solid #4CAF50;">
                    <div style="display: flex; justify-content: space-between; align-items: center;">
                        <div>
                            <strong style="color: #2E7D32;">📊 Report Details</strong>
                            <div style="color: #666; font-size: 14px; margin-top: 5px;">
                                📄 <strong>{filename}</strong><br>
                                💾 Size: {file_size_kb:.1f} KB<br>
                                🏷️ Type: Professional Waste Analysis
                            </div>
                        </div>
                        <div style="text-align: right;">
                            <div style="background: #4CAF50; color: white; padding: 5px 10px; border-radius: 15px; font-size: 12px;">
                                ✅ READY
                            </div>
                        </div>
                    </div>
                </div>
                
                <!-- Action Buttons -->
                <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px; margin-bottom: 20px;">
                    <a href="data:application/pdf;base64,{b64}" download="{filename}" 
                       style="background: linear-gradient(135deg, #4CAF50, #2E7D32); color: white; padding: 12px; 
                              text-decoration: none; border-radius: 6px; text-align: center; font-weight: bold;
                              transition: transform 0.2s; display: block;">
                       📥 Download PDF
                    </a>
                    <button onclick="togglePreview()" 
                            style="background: linear-gradient(135deg, #FF9800, #F57C00); color: white; padding: 12px; 
                                   border: none; border-radius: 6px; text-align: center; font-weight: bold;
                                   cursor: pointer; transition: transform 0.2s;">
                       👁️ Toggle Preview
                    </button>
                </div>
                
                <!-- Preview Section -->
                <div id="previewSection" style="display: none;">
                    <div style="background: white; padding: 15px; border-radius: 8px; border: 1px solid #ddd;">
                        <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 10px;">
                            <strong style="color: #2E7D32;">🔍 Live Preview</strong>
                            <span style="color: #666; font-size: 12px;">Scroll to navigate • Click to interact</span>
                        </div>
                        <iframe src="data:application/pdf;base64,{b64}#toolbar=1&navpanes=1" 
                                width="100%" 
                                height="500px" 
                                style="border: 1px solid #ddd; border-radius: 5px;">
                        </iframe>
                    </div>
                </div>
                
                <!-- Quick Actions -->
                <div style="background: #e8f5e8; padding: 15px; border-radius: 8px; margin-top: 15px;">
                    <strong style="color: #2E7D32;">🚀 Quick Actions</strong>
                    <div style="display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 8px; margin-top: 10px;">
                        <a href="data:application/pdf;base64,{b64}" download="{filename.replace('.pdf', '_compact.pdf')}" 
                           style="background: white; color: #2E7D32; padding: 8px; text-decoration: none; 
                                  border-radius: 4px; text-align: center; font-size: 12px; border: 1px solid #4CAF50;">
                           💾 Save Copy
                        </a>
                        <button onclick="window.print()" 
                                style="background: white; color: #2E7D32; padding: 8px; border: 1px solid #4CAF50; 
                                       border-radius: 4px; font-size: 12px; cursor: pointer;">
                           🖨️ Print
                        </button>
                        <button onclick="shareReport()" 
                                style="background: white; color: #2E7D32; padding: 8px; border: 1px solid #4CAF50; 
                                       border-radius: 4px; font-size: 12px; cursor: pointer;">
                           📤 Share
                        </button>
                    </div>
                </div>
            </div>
            
            <script>
            function togglePreview() {{
                var preview = document.getElementById('previewSection');
                if (preview.style.display === 'none') {{
                    preview.style.display = 'block';
                }} else {{
                    preview.style.display = 'none';
                }}
            }}
            
            function shareReport() {{
                alert('Share functionality: Download the PDF and share the file directly. For web apps, this would integrate with sharing APIs.');
            }}
            </script>
            
            <style>
            a:hover, button:hover {{
                transform: translateY(-2px);
                box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            }}
            </style>
            '''
            return HTML(dashboard_html)
        except Exception as e:
            error_html = f'''
            <div style="color: red; padding: 20px; border: 1px solid red; border-radius: 8px; margin: 20px 0; text-align: center;">
                ❌ Error creating PDF dashboard: {str(e)}
            </div>
            '''
            return HTML(error_html)
    
    @staticmethod
    def display_pdf_info(pdf_bytes: bytes, filename: str):
        """Display enhanced PDF file information"""
        size_kb = len(pdf_bytes) / 1024
        print(f"📊 PDF Report Generated:")
        print(f"   📄 Filename: {filename}")
        print(f"   📦 File size: {size_kb:.1f} KB")
        print(f"   🏷️  Pages: 1 (Professional Report)")
        print(f"   🔧 Features: Enhanced Styling, Branding, Analytics")
        print(f"   💾 Download methods:")
        print(f"      • Click download link below")
        print(f"      • Use preview interface")
        print(f"      • Access from Kaggle output tab")
    
    @staticmethod
    def save_pdf_to_output(pdf_bytes: bytes, filename: str) -> str:
        """Save PDF to Kaggle working directory for manual download"""
        try:
            filepath = f"/kaggle/working/{filename}"
            with open(filepath, 'wb') as f:
                f.write(pdf_bytes)
            print(f"💾 PDF saved to output directory: {filepath}")
            return filepath
        except Exception as e:
            raise Exception(f"Failed to save PDF: {str(e)}")
    
    @staticmethod
    def create_simple_preview(pdf_bytes: bytes, filename: str) -> HTML:
        """Create a simple PDF preview for quick viewing"""
        try:
            b64 = base64.b64encode(pdf_bytes).decode()
            
            simple_html = f'''
            <div style="border: 1px solid #ccc; border-radius: 5px; padding: 10px; margin: 10px 0;">
                <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 10px;">
                    <strong>Preview: {filename}</strong>
                    <a href="data:application/pdf;base64,{b64}" download="{filename}" 
                       style="background: #4CAF50; color: white; padding: 5px 10px; text-decoration: none; border-radius: 3px; font-size: 12px;">
                       Download
                    </a>
                </div>
                <iframe src="data:application/pdf;base64,{b64}" 
                        width="100%" 
                        height="400px" 
                        style="border: 1px solid #ddd;">
                </iframe>
            </div>
            '''
            return HTML(simple_html)
        except Exception as e:
            return HTML(f'<div style="color: red;">Preview error: {str(e)}</div>')

Writing EcoSight/utils/pdf_downloader.py


# Memory Bank Module
**Purpose**: Stores user analysis history and profiles for personalized recommendations over time.

**Contains**:

* User profile tracking with analysis statistics
* Historical waste pattern analysis
* Personalized trend data for recycling optimization
* Session history with timestamp tracking


In [11]:
%%writefile EcoSight/memory/memory_bank.py
from typing import Dict, Any, List, Optional
from datetime import datetime, timedelta

class MemoryBank:
    """Long-term memory storage for user profiles and analysis history"""
    
    def __init__(self, storage_path: Optional[str] = None):
        self.storage_path = storage_path
        self.user_profiles = {}
        self.analysis_history = {}
        
    async def store_analysis(self, user_id: str, session_id: str, results: Dict[str, Any]):
        if user_id not in self.analysis_history:
            self.analysis_history[user_id] = []
        
        record = {
            "session_id": session_id,
            "timestamp": datetime.utcnow().isoformat(),
            "results": results,
            "summary": self._extract_summary(results)
        }
        
        self.analysis_history[user_id].append(record)
        await self._update_user_profile(user_id, record)
        
    async def get_user_history(self, user_id: str, days: int = 30) -> List[Dict[str, Any]]:
        if user_id not in self.analysis_history:
            return []
        
        cutoff_date = datetime.utcnow() - timedelta(days=days)
        history = self.analysis_history[user_id]
        
        return [
            record for record in history
            if datetime.fromisoformat(record['timestamp'].replace('Z', '+00:00')) > cutoff_date
        ]
    
    async def get_user_profile(self, user_id: str) -> Dict[str, Any]:
        if user_id not in self.user_profiles:
            return self._create_default_profile(user_id)
        return self.user_profiles[user_id]
    
    async def _update_user_profile(self, user_id: str, new_record: Dict[str, Any]):
        profile = await self.get_user_profile(user_id)
        summary = new_record['summary']
        
        profile['total_analyses'] += 1
        profile['total_items_analyzed'] += summary['total_items']
        
        for category in ['recyclable', 'compost', 'landfill', 'hazardous']:
            current_avg = profile['category_percentages'].get(category, 0)
            new_value = summary.get(f'{category}_percent', 0)
            profile['category_percentages'][category] = (
                (current_avg * (profile['total_analyses'] - 1) + new_value) / 
                profile['total_analyses']
            )
        
        profile['last_analysis'] = new_record['timestamp']
        self.user_profiles[user_id] = profile
    
    def _extract_summary(self, results: Dict[str, Any]) -> Dict[str, Any]:
        classified_items = results.get('classified_items', [])
        stats = results.get('summary_stats', {})
        
        return {
            'total_items': len(classified_items),
            'recyclable_percent': stats.get('recyclable_percent', 0),
            'compost_percent': stats.get('compost_percent', 0),
            'landfill_percent': stats.get('landfill_percent', 0),
            'hazardous_percent': stats.get('hazardous_percent', 0)
        }
    
    def _create_default_profile(self, user_id: str) -> Dict[str, Any]:
        profile = {
            'user_id': user_id,
            'total_analyses': 0,
            'total_items_analyzed': 0,
            'category_percentages': {
                'recyclable': 0,
                'compost': 0,
                'landfill': 0,
                'hazardous': 0
            },
            'created_date': datetime.utcnow().isoformat(),
            'last_analysis': None,
        }
        
        self.user_profiles[user_id] = profile
        return profile

Writing EcoSight/memory/memory_bank.py


# Vision Analysis Agent

**Purpose**: AI-powered image analysis using Gemini Vision to detect waste items in photos.

**Contains**:

* Integration with Gemini Vision API for real AI analysis
* Fallback to mock provider when API unavailable
* Image processing and item detection with confidence scoring
* Observability metrics for performance tracking


In [12]:
%%writefile EcoSight/agents/vision_agent.py
import base64
import json
from typing import List, Dict, Any
from EcoSight.agents.agent_base import AgentContext, create_agent_logger, log_metric

class VisionAnalysisAgent:
    """Agent for analyzing waste items in images using vision AI"""
    
    def __init__(self, vision_provider: Any, use_gemini: bool = False):
        self.name = "vision_analysis"
        self.logger = create_agent_logger(self.name)
        self.vision_provider = vision_provider
        self.use_gemini = use_gemini
        
        if use_gemini:
            print("🔮 Vision Agent configured to use Gemini (if API key available)")
    
    async def execute(self, context: AgentContext, image_data: bytes) -> Dict[str, Any]:
        self.logger.info(f"Analyzing image for session {context.session_id}")
        
        try:
            # Convert image to base64 for API consumption
            image_b64 = base64.b64encode(image_data).decode('utf-8')
            
            if self.use_gemini:
                print("🎯 Attempting Gemini Vision analysis...")
            
            # Call vision provider (Gemini or Mock)
            detections = await self.vision_provider.analyze_image(image_b64)
            
            log_metric(self.logger, "images_processed", 1)
            log_metric(self.logger, "items_detected", len(detections.get('items', [])))
            
            # Enhanced logging
            provider = detections.get('provider', 'mock')
            model_version = detections.get('model_version', 'unknown')
            print(f"✅ Vision analysis complete - Provider: {provider}, Model: {model_version}")
            
            return {
                "session_id": context.session_id,
                "detections": detections,
                "agent": self.name,
                "provider": provider
            }
            
        except Exception as e:
            self.logger.error(f"Vision analysis failed: {str(e)}")
            raise

Writing EcoSight/agents/vision_agent.py


# Waste Classification Agent

**Purpose**: Categorizes detected waste items into recyclable/compost/landfill with location-specific disposal guidance.

**Contains**:

* Waste categorization using database rules
* Location-aware recycling instructions
* Statistical analysis of waste composition
* Recycling label generation for clear disposal


In [13]:
%%writefile EcoSight/agents/classification_agent.py
from typing import Dict, Any, List
from EcoSight.agents.agent_base import AgentContext, create_agent_logger, log_metric

class WasteClassificationAgent:
    """Agent for classifying waste items and determining disposal methods"""
    
    def __init__(self, waste_db_tool: Any, location_tool: Any):
        self.name = "waste_classification"
        self.logger = create_agent_logger(self.name)
        self.waste_db = waste_db_tool
        self.location_tool = location_tool
    
    async def execute(self, context: AgentContext, vision_results: Dict[str, Any]) -> Dict[str, Any]:
        self.logger.info(f"Classifying items for session {context.session_id}")
        
        classified_items = []
        
        # Process each detected item
        for item in vision_results['detections'].get('items', []):
            # Classify using waste database
            classification = await self.waste_db.classify_item(
                item['name'], 
                context.location
            )
            
            # Get location-specific disposal instructions
            disposal_info = await self.location_tool.get_disposal_instructions(
                classification['category'],
                context.location
            )
            
            classified_item = {
                **item,
                "classification": classification,
                "disposal_instructions": disposal_info,
                "recycling_label": self._generate_recycling_label(classification)
            }
            classified_items.append(classified_item)
        
        # Calculate summary statistics
        stats = self._calculate_classification_stats(classified_items)
        
        log_metric(self.logger, "items_classified", len(classified_items))
        log_metric(self.logger, "recyclable_percent", stats['recyclable_percent'])
        
        return {
            "session_id": context.session_id,
            "classified_items": classified_items,
            "summary_stats": stats,
            "agent": self.name
        }
    
    def _generate_recycling_label(self, classification: Dict[str, Any]) -> str:
        category = classification['category']
        if category == 'recyclable':
            return f"RECYCLE_{classification.get('material', 'MIXED')}"
        elif category == 'compost':
            return "COMPOST"
        elif category == 'hazardous':
            return "HAZARDOUS"
        else:
            return "LANDFILL"
    
    def _calculate_classification_stats(self, items: List[Dict]) -> Dict[str, float]:
        categories = [item['classification']['category'] for item in items]
        total = len(categories)
        
        if total == 0:
            return {'recyclable_percent': 0, 'compost_percent': 0, 
                   'landfill_percent': 0, 'hazardous_percent': 0}
        
        return {
            'recyclable_percent': (categories.count('recyclable') / total) * 100,
            'compost_percent': (categories.count('compost') / total) * 100,
            'landfill_percent': (categories.count('landfill') / total) * 100,
            'hazardous_percent': (categories.count('hazardous') / total) * 100
        }

Writing EcoSight/agents/classification_agent.py


# Reporting & Education Agent

**Purpose**: Generates comprehensive waste analysis reports with PDF generation and personalized educational content.

**Contains**:

* PDF report generation with environmental impact metrics
* Personalized waste management tips based on user history
* Environmental savings calculations (CO₂, water, energy)
* Integration with memory bank for historical insights


In [14]:
%%writefile EcoSight/agents/reporting_agent.py
from typing import Dict, Any, List
from datetime import datetime
from EcoSight.agents.agent_base import AgentContext, create_agent_logger, log_metric
from EcoSight.tools.pdf_generator import PDFGenerator
from EcoSight.utils.pdf_downloader import PDFDownloader

class ReportingEducationAgent:
    """Agent for generating reports and educational content with PDF support"""
    
    def __init__(self, memory_bank: Any):
        self.name = "reporting_education"
        self.logger = create_agent_logger(self.name)
        self.memory_bank = memory_bank
        self.pdf_generator = PDFGenerator()
    
    async def execute(self, context: AgentContext, classification_results: Dict[str, Any]) -> Dict[str, Any]:
        self.logger.info(f"Generating report for session {context.session_id}")
        
        # Store results in memory for historical tracking
        await self.memory_bank.store_analysis(
            context.user_id,
            context.session_id,
            classification_results
        )
        
        # Get user history for personalized insights
        user_history = await self.memory_bank.get_user_history(context.user_id)
        
        # Generate standard report
        report = await self._generate_comprehensive_report(
            classification_results,
            user_history,
            context
        )
        
        # Generate PDF report
        pdf_bytes = await self.generate_pdf_report(report, context.session_id)
        
        # Save PDF to file system
        filename = f"EcoSight_report_{context.session_id}.pdf"
        saved_path = PDFDownloader.save_pdf_to_output(pdf_bytes, filename)
        print(f"💾 PDF saved to: {saved_path}")
        
        # Add PDF data to report
        report['pdf_bytes'] = pdf_bytes
        report['pdf_filename'] = filename
        report['saved_path'] = saved_path
        
        log_metric(self.logger, "reports_generated", 1)
        log_metric(self.logger, "environmental_savings_kg", report['environmental_impact']['co2_saved_kg'])
        
        return report
    
    async def generate_pdf_report(self, report_data: Dict[str, Any], session_id: str) -> bytes:
        """Generate PDF version of the report"""
        self.logger.info(f"Generating PDF report for session {session_id}")
        
        try:
            pdf_bytes = await self.pdf_generator.generate_EcoSight_report(report_data, session_id)
            log_metric(self.logger, "pdf_reports_generated", 1)
            return pdf_bytes
        except Exception as e:
            self.logger.error(f"PDF generation failed: {str(e)}")
            raise
    
    async def _generate_comprehensive_report(self, classification_results: Dict, user_history: List[Dict], context: AgentContext) -> Dict[str, Any]:
        stats = classification_results['summary_stats']
        items = classification_results['classified_items']
        
        # Calculate environmental impact
        environmental_impact = self._calculate_environmental_impact(items)
        
        # Generate personalized tips based on current stats and history
        personalized_tips = self._generate_personalized_tips(stats, user_history)
        
        return {
            "session_id": context.session_id,
            "timestamp": datetime.utcnow().isoformat(),
            "summary": {
                "total_items": len(items),
                "recyclable_percent": stats['recyclable_percent'],
                "compost_percent": stats['compost_percent'],
                "landfill_percent": stats['landfill_percent']
            },
            "environmental_impact": environmental_impact,
            "personalized_tips": personalized_tips,
            "detailed_breakdown": [
                {
                    "item": item['name'],
                    "category": item['classification']['category'],
                    "disposal_instructions": item['disposal_instructions'],
                    "recycling_label": item.get('recycling_label')
                }
                for item in items
            ],
            "agent": self.name
        }
    
    def _calculate_environmental_impact(self, items: List[Dict]) -> Dict[str, float]:
        recyclable_count = sum(1 for item in items if item['classification']['category'] == 'recyclable')
        
        return {
            'co2_saved_kg': recyclable_count * 0.5,
            'water_saved_liters': recyclable_count * 10,
            'energy_saved_kwh': recyclable_count * 0.3
        }
    
    def _generate_personalized_tips(self, current_stats: Dict, user_history: List[Dict]) -> List[str]:
        tips = []
        
        if current_stats['recyclable_percent'] < 50:
            tips.append("Try to separate more recyclable materials like plastic, paper, and metal.")
        
        if current_stats['compost_percent'] < 20:
            tips.append("Consider composting food waste to reduce landfill usage.")
        
        # Add general educational tips
        tips.extend([
            "Rinse recyclable containers to reduce contamination.",
            "Check local guidelines for specific recycling rules.",
            "Reduce single-use plastics by choosing reusable alternatives."
        ])
        
        return tips

Writing EcoSight/agents/reporting_agent.py


# EcoSight Orchestrator

**Purpose**: Coordinates the multi-agent workflow for sequential and parallel waste analysis processing.

**Contains**:

* Sequential pipeline: Vision → Classification → Reporting
* Parallel processing for multiple images
* Session management and error handling
* Context passing between all agents

In [15]:
%%writefile EcoSight/orchestration/orchestrator.py
import asyncio
from typing import Dict, Any, List
from datetime import datetime
import uuid

from EcoSight.agents.vision_agent import VisionAnalysisAgent
from EcoSight.agents.classification_agent import WasteClassificationAgent
from EcoSight.agents.reporting_agent import ReportingEducationAgent
from EcoSight.agents.agent_base import AgentContext

class EcoSightOrchestrator:
    """Orchestrates the multi-agent workflow for waste analysis"""
    
    def __init__(self, vision_provider, waste_db, location_finder, memory_bank, use_gemini_vision: bool = False):
        self.vision_agent = VisionAnalysisAgent(vision_provider, use_gemini=use_gemini_vision)
        self.classification_agent = WasteClassificationAgent(waste_db, location_finder)
        self.reporting_agent = ReportingEducationAgent(memory_bank)
        
    async def process_single_image(self, image_data: bytes, user_id: str = None, 
                                 location: str = None) -> Dict[str, Any]:
        """Sequential workflow: Vision → Classification → Reporting"""
        
        session_id = str(uuid.uuid4())
        context = AgentContext(
            session_id=session_id,
            user_id=user_id or "anonymous",
            location=location,
            metadata={"timestamp": datetime.utcnow().isoformat()}
        )
        
        try:
            print("🔍 Starting Vision Analysis...")
            vision_results = await self.vision_agent.execute(context, image_data)
            print(f"   Detected {len(vision_results['detections'].get('items', []))} items")
            
            print("🏷️ Starting Waste Classification...")
            classification_results = await self.classification_agent.execute(context, vision_results)
            print(f"   Classified {len(classification_results['classified_items'])} items")
            
            print("📊 Generating Report & PDF...")
            final_report = await self.reporting_agent.execute(context, classification_results)
            print("   PDF report generated successfully")
            
            return {
                "success": True,
                "session_id": session_id,
                "report": final_report,
            }
            
        except Exception as e:
            print(f"❌ Pipeline error: {str(e)}")
            return {
                "success": False,
                "session_id": session_id,
                "error": str(e),
            }
    
    async def process_multiple_images(self, images_data: List[bytes], user_id: str = None,
                                    location: str = None) -> Dict[str, Any]:
        """Parallel image processing workflow"""
        
        session_id = str(uuid.uuid4())
        tasks = []
        
        print(f"🔄 Processing {len(images_data)} images in parallel...")
        
        for i, image_data in enumerate(images_data):
            task_context = AgentContext(
                session_id=f"{session_id}_{i}",
                user_id=user_id,
                location=location,
                metadata={"image_index": i}
            )
            
            task = asyncio.create_task(
                self._process_single_image_parallel(image_data, task_context)
            )
            tasks.append(task)
        
        # Wait for all tasks to complete
        results = await asyncio.gather(*tasks, return_exceptions=True)
        successful_results = [r for r in results if not isinstance(r, Exception)]
        
        return {
            "session_id": session_id,
            "total_images": len(images_data),
            "successful_processing": len(successful_results),
            "results": successful_results
        }
    
    async def _process_single_image_parallel(self, image_data: bytes, context: AgentContext) -> Dict[str, Any]:
        """Process single image in parallel workflow"""
        vision_results = await self.vision_agent.execute(context, image_data)
        classification_results = await self.classification_agent.execute(context, vision_results)
        return classification_results

Writing EcoSight/orchestration/orchestrator.py


# Dependencies Installation

**Purpose**: Installs required Python packages for the EcoSight system to function properly.

**Contains**:

* ReportLab for PDF generation and professional reporting
* Google GenAI for Gemini Vision API integration
* Core Python libraries for async operations and data handling

In [16]:
# Install required packages for EcoSight
print("📦 Installing EcoSight dependencies...")

!pip install reportlab
!pip install google-genai

print("✅ Dependencies installed successfully!")
print("📚 Required packages: reportlab, asyncio, base64, typing")

📦 Installing EcoSight dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.3 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
✅ Dependencies installed successfully!
📚 Required packages: reportlab, asyncio, base64, typing


# TrashNet Image Loader 

**Purpose**: Loads real waste images from TrashNet dataset for testing and validation.

**Contains**:

* Multiple dataset path detection for robust loading
* Image preprocessing and format conversion
* Category listing with image counts
* Error handling for missing datasets

In [17]:
# Robust TrashNet Image Loader for Correct Dataset Structure
import os
from PIL import Image
import random

def load_trashnet_sample(category=None):
    """Load a random sample image from TrashNet dataset"""
    
    # Try multiple possible dataset locations (correct TrashNet structure)
    possible_paths = [
        "EcoSight/data/trashnet/data/dataset",  # Copied structure
        "EcoSight/data/trashnet/dataset",       # Alternative copy
        "EcoSight/data/trashnet",               # Direct copy
        "/kaggle/input/trashnet/data/dataset",     # Original location
        "/kaggle/input/trashnet/data",             # Alternative original
        "/kaggle/input/trashnet/dataset",          # Another alternative
        "/kaggle/input/trashnet",                  # Root location
        "/kaggle/input/garbage-classification/Garbage classification/Garbage classification",  # Fallback
    ]
    
    base_path = None
    for path in possible_paths:
        if os.path.exists(path):
            base_path = path
            print(f"📁 Using dataset from: {path}")
            break
    
    if not base_path:
        print("❌ TrashNet dataset not found in any location")
        print("💡 Using mock data for demonstration")
        return None
    
    # TrashNet categories (correct order from original dataset)
    categories = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
    
    # If no category specified, pick random one
    if category is None:
        category = random.choice(categories)
    elif category not in categories:
        print(f"⚠️ Category '{category}' not found. Available: {categories}")
        category = random.choice(categories)
    
    # Try to find the category directory in various possible structures
    category_paths_to_try = [
        os.path.join(base_path, category),
        os.path.join(base_path, 'data', 'dataset', category),
        os.path.join(base_path, 'dataset', category),
    ]
    
    category_path = None
    for test_path in category_paths_to_try:
        if os.path.exists(test_path):
            category_path = test_path
            break
    
    if not category_path:
        print(f"❌ Could not find category '{category}' in dataset")
        print(f"💡 Available directories in {base_path}:")
        try:
            items = os.listdir(base_path)
            dirs = [item for item in items if os.path.isdir(os.path.join(base_path, item))]
            print(f"   {dirs}")
        except:
            print("   Could not list directory contents")
        return None
    
    # Get all image files in category
    try:
        image_files = [f for f in os.listdir(category_path) 
                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    except FileNotFoundError:
        print(f"❌ Category directory not found: {category_path}")
        return None
    
    if not image_files:
        print(f"❌ No images found in {category} category at {category_path}")
        return None
    
    # Select random image
    selected_image = random.choice(image_files)
    image_path = os.path.join(category_path, selected_image)
    
    try:
        # Load and convert to bytes (matching VisionAgent input format)
        with Image.open(image_path) as img:
            # Convert to RGB if needed
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Resize if too large (for efficiency)
            if img.size[0] > 800 or img.size[1] > 800:
                img.thumbnail((800, 800))
            
            # Convert to bytes
            import io
            img_bytes = io.BytesIO()
            img.save(img_bytes, format='JPEG', quality=85)
            image_data = img_bytes.getvalue()
        
        print(f"📸 Loaded TrashNet image: {category}/{selected_image}")
        print(f"📏 Image size: {img.size}, Format: JPEG")
        
        return image_data, category, selected_image
        
    except Exception as e:
        print(f"❌ Error loading image {image_path}: {e}")
        return None

def list_trashnet_categories():
    """List all available categories in TrashNet dataset"""
    possible_paths = [
        "EcoSight/data/trashnet/data/dataset",
        "EcoSight/data/trashnet/dataset", 
        "EcoSight/data/trashnet",
        "/kaggle/input/trashnet/data/dataset",
        "/kaggle/input/trashnet/data",
        "/kaggle/input/trashnet/dataset",
        "/kaggle/input/trashnet",
    ]
    
    categories = []
    base_path = None
    
    for path in possible_paths:
        if os.path.exists(path):
            base_path = path
            break
    
    if base_path:
        print(f"📁 Dataset base path: {base_path}")
        try:
            # Try to list directories at base path
            items = os.listdir(base_path)
            potential_categories = [item for item in items 
                                  if os.path.isdir(os.path.join(base_path, item))]
            
            # Filter for actual TrashNet categories
            trashnet_categories = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
            categories = [cat for cat in potential_categories if cat in trashnet_categories]
            
            if not categories:
                # Maybe we're at a higher level, check subdirectories
                for item in items:
                    sub_path = os.path.join(base_path, item)
                    if os.path.isdir(sub_path):
                        sub_items = os.listdir(sub_path)
                        for sub_item in sub_items:
                            if sub_item in trashnet_categories and sub_item not in categories:
                                categories.append(sub_item)
            
            print("📋 Available TrashNet categories:")
            for cat in sorted(categories):
                # Find the actual path to count images
                for test_base in possible_paths:
                    test_paths = [
                        os.path.join(test_base, cat),
                        os.path.join(test_base, 'data', 'dataset', cat),
                        os.path.join(test_base, 'dataset', cat),
                    ]
                    for test_path in test_paths:
                        if os.path.exists(test_path):
                            try:
                                image_count = len([f for f in os.listdir(test_path) 
                                                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                                print(f"   • {cat}: {image_count} images")
                                break
                            except:
                                print(f"   • {cat}: found but cannot access")
                            break
                    else:
                        continue
                    break
                else:
                    print(f"   • {cat}: directory not found")
                    
        except Exception as e:
            print(f"❌ Error reading dataset structure: {e}")
    else:
        print("❌ TrashNet dataset not found")
        print("💡 Please add: 'trashnet' by yangyang111 via '+ Add Data'")
        categories = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']  # Default list
    
    return categories

print("🔄 TrashNet utilities loaded with correct dataset structure!")

🔄 TrashNet utilities loaded with correct dataset structure!



# EcoSight System Initialization

**Purpose**: Bootstraps the complete multi-agent system with all components and dependencies.

**Contains**:

* Automatic Gemini API key detection and provider selection
* Component initialization (Vision, Classification, Reporting, Memory)
* Orchestrator setup with agent coordination
* System health check and status reporting


In [18]:
# EcoSight System Initialization
print("🚀 Initializing EcoSight Multi-Agent System...")

import sys
import os

# Ensure EcoSight is in path
sys.path.append('/kaggle/working/EcoSight')

# Import all components
from EcoSight.tools.vision_provider import MockVisionProvider, GeminiVisionProvider
from EcoSight.tools.waste_db import WasteDB
from EcoSight.tools.location_finder import LocationFinder
from EcoSight.tools.pdf_generator import PDFGenerator
from EcoSight.memory.memory_bank import MemoryBank
from EcoSight.orchestration.orchestrator import EcoSightOrchestrator
from EcoSight.utils.pdf_downloader import PDFDownloader

# Initialize components with error handling
try:
    # Try Gemini first if API key is available
    gemini_api_key = os.getenv("GEMINI_API_KEY")
    if gemini_api_key and gemini_api_key != "MOCK_MODE_NO_VALID_KEY" and gemini_api_key.startswith("AIza"):
        print("🔮 Using REAL Gemini Vision Provider")
        vision_provider = GeminiVisionProvider()
        use_gemini = True
    else:
        print("🎭 Using Mock Vision Provider (no valid Gemini API key)")
        vision_provider = MockVisionProvider()
        use_gemini = False
except Exception as e:
    print(f"🎭 Using Mock Vision Provider (Gemini initialization failed: {e})")
    vision_provider = MockVisionProvider()
    use_gemini = False

# Initialize other components
waste_db = WasteDB()
location_finder = LocationFinder()
memory_bank = MemoryBank()

# Create orchestrator
orchestrator = EcoSightOrchestrator(
    vision_provider=vision_provider,
    waste_db=waste_db,
    location_finder=location_finder,
    memory_bank=memory_bank,
    use_gemini_vision=use_gemini
)

print("✅ EcoSight system initialized successfully!")
print("📋 Components ready:")
print("   - Vision Analysis Agent")
print("   - Waste Classification Agent") 
print("   - Reporting & Education Agent (with PDF)")
print("   - Memory Bank")
print("   - PDF Generator")
print(f"   - Vision Provider: {'Gemini' if use_gemini else 'Mock'}")
print("   - TrashNet dataset integrated")

# Display Gemini status
if use_gemini:
    print("🎉 Real AI detection: ENABLED")
else:
    print("💡 Real AI detection: DISABLED - Add Gemini API key to Kaggle Secrets")

🚀 Initializing EcoSight Multi-Agent System...
🔮 Using REAL Gemini Vision Provider
❌ Model gemini-1.5-flash failed: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateCon...
❌ Model gemini-1.5-pro failed: 404 models/gemini-1.5-pro is not found for API version v1beta, or is not supported for generateConte...
❌ Model gemini-pro failed: 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. ...
❌ Model gemini-pro-vision failed: 404 models/gemini-pro-vision is not found for API version v1beta, or is not supported for generateCo...
🔍 Listing available models...
📋 Available: models/gemini-2.5-pro-preview-03-25
📋 Available: models/gemini-2.5-flash
✅ Using model: models/gemini-2.5-flash
✅ Gemini Vision initialized with model: models/gemini-2.5-flash
🔮 Vision Agent configured to use Gemini (if API key available)
✅ EcoSight system initialized successfully!
📋 Components ready:
   - Vision Analysis Agent
   -

# Dataset Verification Cell

**Purpose**: Validates TrashNet dataset access and provides detailed category statistics.

**Contains**:

* Dataset structure analysis and path verification
* Category-wise image counts across train/val/test splits
* Comprehensive dataset summary with totals
* Debug information for troubleshooting missing datasets

In [19]:
# =============================================================================
# 🧪 TEST CELL: Dataset Verification 
# =============================================================================

print("🧪 Verifying TrashNet Dataset Access...")
print("1. Checking available categories...")

import os
import glob

def find_actual_categories():
    """Find the actual waste categories with images"""
    
    # The dataset path
    base_path = '/kaggle/input/trashnet/trashnet'
    
    if not os.path.exists(base_path):
        print(f"❌ Dataset path not found: {base_path}")
        return None, []
    
    print(f"📁 Dataset path: {base_path}")
    
    # List ALL items in the base path
    all_items = os.listdir(base_path)
    print(f"📦 Main folders: {all_items}")
    
    # Check each subfolder (train, val, test) for categories
    found_categories = {}
    
    for split in ['train', 'val', 'test']:
        split_path = os.path.join(base_path, split)
        if os.path.exists(split_path):
            print(f"\n🔍 Checking {split} folder:")
            
            # List categories in this split
            categories_in_split = os.listdir(split_path)
            print(f"   Categories in {split}: {categories_in_split}")
            
            for category in categories_in_split:
                category_path = os.path.join(split_path, category)
                if os.path.isdir(category_path):
                    # Count images in this category
                    image_files = glob.glob(os.path.join(category_path, "*.jpg")) + \
                                 glob.glob(os.path.join(category_path, "*.jpeg")) + \
                                 glob.glob(os.path.join(category_path, "*.png"))
                    
                    if image_files:
                        if category not in found_categories:
                            found_categories[category] = {'total': 0, 'train': 0, 'val': 0, 'test': 0}
                        found_categories[category][split] = len(image_files)
                        found_categories[category]['total'] += len(image_files)
                        print(f"   ✅ {category}: {len(image_files)} images")
    
    return base_path, found_categories

# Find the actual categories
dataset_path, categories_dict = find_actual_categories()

if categories_dict:
    print(f"\n🎉 SUCCESS: Found TrashNet dataset with categories!")
    print("📋 Available TrashNet categories (with split counts):")
    
    for category in sorted(categories_dict.keys()):
        counts = categories_dict[category]
        print(f"   ✅ {category}:")
        print(f"      📊 Total: {counts['total']} images")
        if counts['train'] > 0:
            print(f"      🏋️‍♂️ Train: {counts['train']} images")
        if counts['val'] > 0:
            print(f"      📈 Val: {counts['val']} images")
        if counts['test'] > 0:
            print(f"      🧪 Test: {counts['test']} images")
    
    # Calculate totals
    total_images = sum(cat_info['total'] for cat_info in categories_dict.values())
    total_train = sum(cat_info['train'] for cat_info in categories_dict.values())
    total_val = sum(cat_info['val'] for cat_info in categories_dict.values())
    total_test = sum(cat_info['test'] for cat_info in categories_dict.values())
    
    print(f"\n📊 DATASET SUMMARY:")
    print(f"   🎯 Categories: {len(categories_dict)}")
    print(f"   🖼️  Total images: {total_images}")
    print(f"   🏋️‍♂️ Training images: {total_train}")
    print(f"   📈 Validation images: {total_val}")
    print(f"   🧪 Test images: {total_test}")
    
else:
    print("\n❌ Could not find any categories!")
    if dataset_path:
        print(f"🔍 Debugging structure of: {dataset_path}")
        print(f"\n📂 Complete contents:")
        for item in os.listdir(dataset_path):
            item_path = os.path.join(dataset_path, item)
            if os.path.isdir(item_path):
                print(f"\n   📁 {item}/:")
                subitems = os.listdir(item_path)
                for subitem in subitems:
                    subitem_path = os.path.join(item_path, subitem)
                    if os.path.isdir(subitem_path):
                        image_files = glob.glob(os.path.join(subitem_path, "*.jpg")) + \
                                     glob.glob(os.path.join(subitem_path, "*.jpeg")) + \
                                     glob.glob(os.path.join(subitem_path, "*.png"))
                        print(f"      📁 {subitem}/: {len(image_files)} images")
                        if len(image_files) > 0:
                            for img in image_files[:2]:  # Show first 2 images
                                print(f"         🖼️  {os.path.basename(img)}")

print("\n" + "="*60)
print("✅ Dataset verification completed!")

🧪 Verifying TrashNet Dataset Access...
1. Checking available categories...
📁 Dataset path: /kaggle/input/trashnet/trashnet
📦 Main folders: ['val', 'test', 'train']

🔍 Checking train folder:
   Categories in train: ['metal', 'glass', 'paper', 'trash', 'cardboard', 'plastic']
   ✅ metal: 287 images
   ✅ glass: 350 images
   ✅ paper: 415 images
   ✅ trash: 95 images
   ✅ cardboard: 282 images
   ✅ plastic: 337 images

🔍 Checking val folder:
   Categories in val: ['metal', 'glass', 'paper', 'trash', 'cardboard', 'plastic']
   ✅ metal: 82 images
   ✅ glass: 100 images
   ✅ paper: 118 images
   ✅ trash: 27 images
   ✅ cardboard: 80 images
   ✅ plastic: 96 images

🔍 Checking test folder:
   Categories in test: ['metal', 'glass', 'paper', 'trash', 'cardboard', 'plastic']
   ✅ metal: 41 images
   ✅ glass: 51 images
   ✅ paper: 61 images
   ✅ trash: 15 images
   ✅ cardboard: 41 images
   ✅ plastic: 49 images

🎉 SUCCESS: Found TrashNet dataset with categories!
📋 Available TrashNet categories (wit


# Single Image Test Cell

**Purpose**: Tests the complete EcoSight pipeline with real TrashNet waste images.

**Contains**:

* Color issue fixes for PDF generation
* Random image selection from TrashNet categories
* Complete pipeline testing (Vision → Classification → Reporting → PDF)
* Error handling and debugging information

In [20]:
# =============================================================================
# 🧪 SINGLE TRASHNET IMAGE TEST 
# =============================================================================

print("🧪 Testing EcoSight with Real TrashNet Waste Images...")

import os
import random
import glob
from PIL import Image
import io
import reportlab.lib.colors as colors

# COMPREHENSIVE FIX FOR COLOR ISSUE
def fix_color_issues():
    """Fix all color-related issues in PDF generation"""
    try:
        # Fix 1: Add proper toHex method that handles float values
        if not hasattr(colors.Color, 'toHex'):
            def color_toHex(self):
                # Convert float RGB values (0-1) to integer (0-255)
                r = int(self.red * 255)
                g = int(self.green * 255) 
                b = int(self.blue * 255)
                return "#%02x%02x%02x" % (r, g, b)
            colors.Color.toHex = color_toHex
            print("✅ Fixed Color.toHex with float-to-int conversion")
        
        # Fix 2: Also patch any other color conversion issues
        import reportlab.lib.utils as utils
        original_getBytes = getattr(utils, 'getBytes', None)
        
        if original_getBytes:
            def safe_getBytes(s):
                try:
                    return original_getBytes(s)
                except:
                    return str(s).encode('utf-8')
            utils.getBytes = safe_getBytes
            print("✅ Patched getBytes for safe string conversion")
            
    except Exception as e:
        print(f"⚠️  Color fix warning: {e}")

# Apply the color fixes
fix_color_issues()

def get_trashnet_categories():
    """Get available waste categories with their image counts"""
    base_path = '/kaggle/input/trashnet/trashnet'
    
    if not os.path.exists(base_path):
        print("❌ Could not find TrashNet dataset")
        return None, {}
    
    categories = set()
    
    for split in ['train', 'val', 'test']:
        split_path = os.path.join(base_path, split)
        if os.path.exists(split_path):
            categories.update([d for d in os.listdir(split_path) if os.path.isdir(os.path.join(split_path, d))])
    
    category_counts = {}
    for category in categories:
        total_images = 0
        for split in ['train', 'val', 'test']:
            split_path = os.path.join(base_path, split, category)
            if os.path.exists(split_path):
                image_files = glob.glob(os.path.join(split_path, "*.jpg")) + \
                             glob.glob(os.path.join(split_path, "*.jpeg")) + \
                             glob.glob(os.path.join(split_path, "*.png"))
                total_images += len(image_files)
        category_counts[category] = total_images
    
    return base_path, category_counts

def get_random_image(category=None):
    """Get a random image from the dataset and convert to bytes"""
    base_path = '/kaggle/input/trashnet/trashnet'
    
    _, categories_dict = get_trashnet_categories()
    
    if not categories_dict:
        return None, None, None
    
    if category and category in categories_dict:
        selected_category = category
    else:
        selected_category = random.choice(list(categories_dict.keys()))
    
    for split in ['train', 'val', 'test']:
        category_path = os.path.join(base_path, split, selected_category)
        if os.path.exists(category_path):
            image_files = glob.glob(os.path.join(category_path, "*.jpg")) + \
                         glob.glob(os.path.join(category_path, "*.jpeg")) + \
                         glob.glob(os.path.join(category_path, "*.png"))
            if image_files:
                selected_image_path = random.choice(image_files)
                
                with Image.open(selected_image_path) as img:
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                    
                    img_bytes = io.BytesIO()
                    img.save(img_bytes, format='JPEG')
                    image_data = img_bytes.getvalue()
                
                return image_data, selected_category, selected_image_path
    
    return None, selected_category, None

# Get available categories
dataset_path, categories_dict = get_trashnet_categories()

if categories_dict:
    print("🎉 TrashNet Dataset Found!")
    print("📋 Available TrashNet categories:")
    for category in sorted(categories_dict.keys()):
        print(f"   ✅ {category}: {categories_dict[category]} images")
    
    total_images = sum(categories_dict.values())
    print(f"📊 TOTAL: {total_images} images")
else:
    print("❌ No waste categories found!")

async def test_trashnet_pipeline(category=None, use_mock=True, skip_pdf=False):
    """Test the pipeline with a random image from TrashNet"""
    print(f"\n🚀 Starting pipeline test with TrashNet image...")
    
    image_data, selected_category, image_path = get_random_image(category)
    
    if not image_data:
        print("❌ Could not load any images!")
        return
    
    print(f"🎲 Selected category: {selected_category}")
    print(f"🖼️  Selected image: {os.path.basename(image_path)}")
    
    try:
        # Force mock mode to avoid API issues
        original_use_gemini = getattr(orchestrator, 'use_gemini', True)
        
        if use_mock:
            orchestrator.use_gemini = False
            print("🔧 Using mock mode to avoid API issues")
        
        # Skip PDF generation if requested
        original_generate_pdf = getattr(orchestrator, 'generate_pdf', True)
        if skip_pdf:
            orchestrator.generate_pdf = False
            print("🔧 Skipping PDF generation to avoid color issues")
        
        print("🔍 Processing image through EcoSight pipeline...")
        
        # Call the orchestrator's process_single_image method
        result = await orchestrator.process_single_image(
            image_data=image_data,
            user_id=f"trashnet_test_{selected_category}",
            location="NYC"
        )
        
        # Restore original settings
        orchestrator.use_gemini = original_use_gemini
        if skip_pdf:
            orchestrator.generate_pdf = original_generate_pdf
        
        if result["success"]:
            print(f"✅ Pipeline test successful!")
            report = result["report"]
            
            # Display results
            print(f"\n📊 ANALYSIS RESULTS:")
            print(f"   Items detected: {report['summary']['total_items']}")
            print(f"   Recyclable: {report['summary']['recyclable_percent']:.1f}%")
            print(f"   Compost: {report['summary']['compost_percent']:.1f}%")
            print(f"   Landfill: {report['summary']['landfill_percent']:.1f}%")
            
            if report['detailed_breakdown']:
                print(f"\n🔍 DETECTED ITEMS:")
                for item in report['detailed_breakdown']:
                    print(f"   • {item['item']} → {item['category']}")
            
            print(f"\n💡 PERSONALIZED TIPS:")
            for tip in report['personalized_tips'][:3]:
                print(f"   • {tip}")
                
            # Show PDF info if available
            if 'pdf_bytes' in report:
                pdf_size = len(report['pdf_bytes']) / 1024
                print(f"\n📄 PDF Report: {report['pdf_filename']} ({pdf_size:.1f} KB)")
            else:
                print(f"\n📄 PDF: Not generated (skipped)")
                
            return result
        else:
            print(f"❌ Pipeline failed: {result.get('error', 'Unknown error')}")
            return None
            
    except Exception as e:
        print(f"❌ Pipeline test failed: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# Run tests with different options
print("\n" + "="*60)
if categories_dict:
    print("💡 Available test options:")
    print("   1. await test_trashnet_pipeline()                    # Basic test")
    print("   2. await test_trashnet_pipeline('plastic')           # Specific category") 
    print("   3. await test_trashnet_pipeline(skip_pdf=True)       # Skip PDF generation")
    
    # Test 1: Basic test with PDF
    print(f"\n🎯 Test 1: Basic test with PDF generation...")
    result1 = await test_trashnet_pipeline(use_mock=True, skip_pdf=False)
    
    if result1 and not result1["success"]:
        # Test 2: Skip PDF generation
        print(f"\n🔄 Test 2: Trying without PDF generation...")
        result2 = await test_trashnet_pipeline(use_mock=True, skip_pdf=True)
        
        if result2 and result2["success"]:
            print(f"\n🎉 SUCCESS! Pipeline works without PDF generation.")
            print("💡 The issue is specifically in PDF color conversion.")
        else:
            print(f"\n🔧 Need deeper troubleshooting...")
    
else:
    print("❌ Cannot run tests - no categories available")

# Additional debugging
print("\n" + "="*60)
print("🔧 DEBUGGING INFO:")
print("✅ Color.toHex method fixed with float-to-int conversion")
print("✅ Mock mode enabled to avoid Gemini API issues")
print("💡 If PDF still fails, the issue is in specific color usage in your PDF template")

🧪 Testing EcoSight with Real TrashNet Waste Images...
✅ Fixed Color.toHex with float-to-int conversion
🎉 TrashNet Dataset Found!
📋 Available TrashNet categories:
   ✅ cardboard: 403 images
   ✅ glass: 501 images
   ✅ metal: 410 images
   ✅ paper: 594 images
   ✅ plastic: 482 images
   ✅ trash: 137 images
📊 TOTAL: 2527 images

💡 Available test options:
   1. await test_trashnet_pipeline()                    # Basic test
   2. await test_trashnet_pipeline('plastic')           # Specific category
   3. await test_trashnet_pipeline(skip_pdf=True)       # Skip PDF generation

🎯 Test 1: Basic test with PDF generation...

🚀 Starting pipeline test with TrashNet image...
🎲 Selected category: trash
🖼️  Selected image: trash41.jpg
🔧 Using mock mode to avoid API issues
🔍 Processing image through EcoSight pipeline...
🔍 Starting Vision Analysis...
🎯 Attempting Gemini Vision analysis...
🔮 Calling real Gemini Vision API with model: models/gemini-2.5-flash...
📊 Detected 5 raw items → 5 after enhanced f

# Gemini AI Verification Test
**Purpose**: Validates real Gemini Vision API integration and compares with mock provider.

**Contains**:

* Direct Gemini API testing with real image analysis
* Detection results comparison and confidence scoring
* Raw response preview for debugging
* Fallback handling when API key is unavailable

In [21]:
# =============================================================================
# 🧪 GEMINI AI VERIFICATION TEST
# =============================================================================

print("🧪 Testing Gemini AI Integration...")

import base64  

async def test_gemini_detection():
    """Test if real Gemini detection is working"""
    if use_gemini:
        print("🚀 Testing REAL Gemini Vision detection...")
        # Load a test image
        image_data, category, image_path = get_random_image()
        if image_data:
            try:
                # Test direct Gemini call
                image_b64 = base64.b64encode(image_data).decode('utf-8')
                result = await vision_provider.analyze_image(image_b64)
                
                print(f"✅ Real Gemini detection successful!")
                print(f"   Provider: {result['provider']}")
                print(f"   Model: {result['model_version']}")
                print(f"   Items detected: {len(result['items'])}")
                
                for i, item in enumerate(result['items'][:3], 1):  # Show first 3 items
                    print(f"   {i}. {item['name']} (confidence: {item['confidence']:.2f})")
                    if 'description' in item:
                        print(f"      Description: {item['description'][:60]}...")
                
                if 'raw_response' in result:
                    print(f"   Raw response preview: {result['raw_response'][:100]}...")
                    
            except Exception as e:
                print(f"❌ Real Gemini test failed: {e}")
                print("💡 Falling back to mock mode for this session")
    else:
        print("🎭 Using mock detection - no real Gemini API key configured")
        print("💡 Add your Gemini API key to Kaggle Secrets for real AI detection")

await test_gemini_detection()

🧪 Testing Gemini AI Integration...
🚀 Testing REAL Gemini Vision detection...
🔮 Calling real Gemini Vision API with model: models/gemini-2.5-flash...
📊 Detected 5 raw items → 5 after enhanced filtering
✅ Real Gemini detection successful!
   Provider: gemini
   Model: models/gemini-2.5-flash
   Items detected: 5
   1. paper bag (confidence: 0.88)
      Description: **Material composition:** Paper...
   2. paper bag (confidence: 0.88)
      Description: **Approximate size/condition:** It's a single sheet of paper...
   3. here is (confidence: 0.78)
      Description: Here is a list of all identifiable waste items in the image:...
   Raw response preview: Here is a list of all identifiable waste items in the image:

1.  **Specific type:** Retail advertis...


# Multiple Images Test Cell

**Purpose**: Demonstrates parallel processing capabilities by analyzing multiple TrashNet images simultaneously.

**Contains**:

* Bulk image loading from multiple waste categories
* Parallel processing with orchestrator
* Performance metrics and success rate tracking
* Category-wise analysis and recyclable rate calculation


In [22]:
# =============================================================================
# 🔄 Multiple Images TEST (Optimized)
# =============================================================================

print("🔄 Testing Comprehensive Bulk Processing with Multiple Images Per Category...")

async def test_comprehensive_bulk_processing():
    """Process multiple images from each category for meaningful bulk testing"""
    
    images_per_category = 7  # Process 7 images from each category
    total_target_images = len(categories_dict) * images_per_category
    
    print(f"🎯 Target: Processing {total_target_images} images ({images_per_category} from each category)")
    
    test_images = []
    loaded_info = []
    
    print("📥 Loading multiple images from all categories...")
    
    for category in categories_dict.keys():
        images_loaded = 0
        for i in range(images_per_category):
            image_data, actual_category, image_path = get_random_image(category)
            if image_data:
                test_images.append(image_data)
                loaded_info.append(f"{actual_category}: {os.path.basename(image_path)}")
                images_loaded += 1
        
        print(f"   ✅ {category}: {images_loaded}/{images_per_category} images loaded")
    
    if not test_images:
        print("❌ No images loaded for bulk testing")
        return
    
    print(f"\n📸 Processing {len(test_images)} TrashNet images in parallel...")
    
    try:
        result = await orchestrator.process_multiple_images(
            images_data=test_images,
            user_id=f"comprehensive_bulk_{len(test_images)}_images",
            location="Comprehensive Analysis"
        )
        
        # Check if we had API quota issues
        api_errors = 0
        for batch_result in result['results']:
            if batch_result and 'provider' in batch_result and batch_result['provider'] == 'mock':
                api_errors += 1
        
        if api_errors > 10:  # If more than 10 images used mock provider
            print(f"\n⚠️  API QUOTA EXCEEDED: {api_errors}/{result['total_images']} images used mock data")
            print("💡 Stopping further tests to avoid API limitations")
            return result
        
        print(f"\n✅ BULK PROCESSING COMPLETED!")
        print(f"📊 COMPREHENSIVE RESULTS:")
        print(f"   📸 Total images processed: {result['total_images']}")
        print(f"   ✅ Successful analyses: {result['successful_processing']}")
        
        if result['successful_processing'] > 0:
            # Enhanced statistics
            total_items = 0
            category_stats = {}
            recyclable_sum = 0
            valid_results = 0
            
            for i, batch_result in enumerate(result['results']):
                if batch_result and 'summary_stats' in batch_result:
                    stats = batch_result['summary_stats']
                    total_items += stats.get('total_items', 0)
                    recyclable_sum += stats.get('recyclable_percent', 0)
                    valid_results += 1
                    
                    # Track by category
                    if i < len(loaded_info):
                        category = loaded_info[i].split(':')[0]
                        if category not in category_stats:
                            category_stats[category] = []
                        category_stats[category].append(stats.get('recyclable_percent', 0))
            
            if valid_results > 0:
                avg_recyclable = recyclable_sum / valid_results
                
                print(f"\n📈 DETAILED ANALYSIS:")
                print(f"   🔢 Total items analyzed: {total_items}")
                print(f"   ♻️  Average recyclable rate: {avg_recyclable:.1f}%")
                print(f"   📊 Average items per image: {total_items/valid_results:.1f}")
                
                # Category-wise analysis
                print(f"\n🏷️  CATEGORY PERFORMANCE:")
                for category, percentages in category_stats.items():
                    if percentages:
                        avg_percent = sum(percentages) / len(percentages)
                        print(f"   {category}: {avg_percent:.1f}% recyclable ({len(percentages)} images)")
        
        return result
        
    except Exception as e:
        print(f"❌ Comprehensive bulk processing failed: {str(e)}")
        return None

async def test_large_scale_processing(total_images=25):
    """Process a larger number of random images with API quota checking"""
    
    print(f"\n🎯 LARGE SCALE: Processing {total_images} random TrashNet images...")
    
    test_images = []
    
    for i in range(total_images):
        image_data, category, image_path = get_random_image()
        if image_data:
            test_images.append(image_data)
    
    print(f"✅ Successfully loaded {len(test_images)} random images")
    
    if test_images:
        result = await orchestrator.process_multiple_images(
            images_data=test_images,
            user_id=f"large_scale_{len(test_images)}_images",
            location="Large Scale Test"
        )
        
        # Check API quota usage
        api_errors = 0
        for batch_result in result['results']:
            if batch_result and 'provider' in batch_result and batch_result['provider'] == 'mock':
                api_errors += 1
        
        if api_errors > 5:  # If more than 5 images used mock provider
            print(f"⚠️  API QUOTA EXCEEDED: {api_errors}/{result['total_images']} images used mock data")
            print("💡 Stopping further API calls to avoid limitations")
            return None
        
        if result and result['successful_processing'] > 0:
            print(f"✅ Large scale processing completed!")
            print(f"   Processed: {result['successful_processing']}/{result['total_images']} images successfully")
        
        return result
    return None

# Run the comprehensive test
print("🚀 Starting comprehensive bulk processing...")
result1 = await test_comprehensive_bulk_processing()

# Only run large scale test if we didn't hit API limits
if result1:
    # Check if we should proceed based on API usage
    api_errors_in_first = 0
    for batch_result in result1['results']:
        if batch_result and 'provider' in batch_result and batch_result['provider'] == 'mock':
            api_errors_in_first += 1
    
    if api_errors_in_first <= 5:  # Proceed only if we have API quota left
        print("\n" + "="*50)
        print("🎯 Testing larger scale processing...")
        await test_large_scale_processing(15)  # Reduced from 25 to 15
    else:
        print(f"\n⚠️  SKIPPING LARGE SCALE TEST: {api_errors_in_first} API quota issues detected")
else:
    print("❌ Initial bulk processing failed, skipping large scale test")

print("\n✅ BULK PROCESSING COMPLETED WITH API QUOTA PROTECTION")

🔄 Testing Comprehensive Bulk Processing with Multiple Images Per Category...
🚀 Starting comprehensive bulk processing...
🎯 Target: Processing 42 images (7 from each category)
📥 Loading multiple images from all categories...
   ✅ paper: 7/7 images loaded
   ✅ trash: 7/7 images loaded
   ✅ plastic: 7/7 images loaded
   ✅ cardboard: 7/7 images loaded
   ✅ metal: 7/7 images loaded
   ✅ glass: 7/7 images loaded

📸 Processing 42 TrashNet images in parallel...
🔄 Processing 42 images in parallel...
🎯 Attempting Gemini Vision analysis...
🔮 Calling real Gemini Vision API with model: models/gemini-2.5-flash...
🎯 Attempting Gemini Vision analysis...
🔮 Calling real Gemini Vision API with model: models/gemini-2.5-flash...
🎯 Attempting Gemini Vision analysis...
🔮 Calling real Gemini Vision API with model: models/gemini-2.5-flash...
🎯 Attempting Gemini Vision analysis...
🔮 Calling real Gemini Vision API with model: models/gemini-2.5-flash...
🎯 Attempting Gemini Vision analysis...
🔮 Calling real Gemini

# PDF Generation Test Cell

**Purpose**: Tests professional PDF report generation with interactive preview and download options.

**Contains**:

* PDF downloader functionality verification
* Interactive dashboard with toggle preview
* Multiple download format options
* File management and output directory saving

In [23]:
# =============================================================================
# 📄 PDF GENERATION TEST 
# =============================================================================

print("🔄 Loading PDF downloader...")

# Import the correct PDFDownloader
from EcoSight.utils.pdf_downloader import PDFDownloader

# Verify the import worked
print("✅ PDFDownloader methods available:")
print(f"   - create_interactive_pdf_dashboard: {hasattr(PDFDownloader, 'create_interactive_pdf_dashboard')}")
print(f"   - create_simple_preview: {hasattr(PDFDownloader, 'create_simple_preview')}")
print(f"   - create_download_link: {hasattr(PDFDownloader, 'create_download_link')}")
print(f"   - display_pdf_info: {hasattr(PDFDownloader, 'display_pdf_info')}")
print(f"   - save_pdf_to_output: {hasattr(PDFDownloader, 'save_pdf_to_output')}")

async def test_enhanced_pdf_generation():
    """Test PDF generation with enhanced preview functionality"""
    # Load a TrashNet image for testing
    print("🔄 Loading waste image for enhanced PDF test...")
    image_data, category, image_path = get_random_image('plastic')
    
    if image_data is None:
        print("❌ Failed to load image, trying random category...")
        image_data, category, image_path = get_random_image()
    
    if image_data is None:
        print("❌ Could not load any images for PDF test")
        return
    
    print(f"🎯 Generating enhanced PDF report for: {category} waste")
    print(f"🖼️ Source image: {os.path.basename(image_path)}")
    
    # Process through pipeline
    print("🔍 Processing through EcoSight pipeline...")
    pipeline_result = await orchestrator.process_single_image(
        image_data=image_data,
        user_id=f"enhanced_pdf_test_{category}",
        location="NYC"
    )
    
    if pipeline_result["success"]:
        if 'pdf_bytes' in pipeline_result["report"]:
            report = pipeline_result["report"]
            pdf_bytes = report['pdf_bytes']
            pdf_filename = report['pdf_filename']
            
            # Display enhanced PDF information
            print("✅ Enhanced PDF Generation Successful!")
            PDFDownloader.display_pdf_info(pdf_bytes, pdf_filename)
            
            # Save to output directory
           # saved_path = PDFDownloader.save_pdf_to_output(pdf_bytes, pdf_filename)
            
             #print("\n" + "="*60)
             #print("🎨 ENHANCED PDF PREVIEW OPTIONS:")
             #print("="*60)
            
            # Option 1: Interactive Dashboard (Recommended)
            print("\n1. 🚀 INTERACTIVE DASHBOARD:")
            dashboard = PDFDownloader.create_interactive_pdf_dashboard(pdf_bytes, pdf_filename)
            display(dashboard)
            
            # Option 2: Simple Preview
             # print("\n2. 👀 SIMPLE PREVIEW:")
             # simple_preview = PDFDownloader.create_simple_preview(pdf_bytes, pdf_filename)
             # display(simple_preview)
            
            #Option 3: Standard Download Link
             # print("\n3. 📥 STANDARD DOWNLOAD:")
             # download_link = PDFDownloader.create_download_link(pdf_bytes, pdf_filename, "📥 Download Enhanced Report")
              #display(download_link)
            
        else:
            print("❌ PDF generation failed - no PDF data in report")
    else:
        print("❌ Pipeline processing failed")
        print(f"   Error: {pipeline_result.get('error', 'Unknown error')}")

# Run enhanced PDF generation test
print("🚀 Starting enhanced PDF generation test...")
await test_enhanced_pdf_generation()

🔄 Loading PDF downloader...
✅ PDFDownloader methods available:
   - create_interactive_pdf_dashboard: True
   - create_simple_preview: True
   - create_download_link: True
   - display_pdf_info: True
   - save_pdf_to_output: True
🚀 Starting enhanced PDF generation test...
🔄 Loading waste image for enhanced PDF test...
🎯 Generating enhanced PDF report for: plastic waste
🖼️ Source image: plastic149.jpg
🔍 Processing through EcoSight pipeline...
🔍 Starting Vision Analysis...
🎯 Attempting Gemini Vision analysis...
🔮 Calling real Gemini Vision API with model: models/gemini-2.5-flash...
❌ Gemini Vision API error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-fl

# Enhanced PDF Report Generation Test

**Purpose**: Demonstrates comprehensive PDF reporting with multiple preview options and styling.

**Contains**:

* Multiple PDF preview interfaces (dashboard, simple, download)
* Environmental impact reporting with CO₂ savings
* Professional styling and branding
* File management and output organization

In [24]:
# =============================================================================
# 📄 ENHANCED PDF REPORT GENERATION TEST (WITH PREVIEW)
# =============================================================================

print("📄 Testing Enhanced PDF Report Generation with Preview...")

async def test_enhanced_pdf_generation():
    """Test PDF generation with enhanced preview functionality"""
    # Load a TrashNet image for testing
    print("🔄 Loading waste image for enhanced PDF test...")
    image_data, category, image_path = get_random_image('plastic')
    
    if image_data is None:
        print("❌ Failed to load image, trying random category...")
        image_data, category, image_path = get_random_image()
    
    if image_data is None:
        print("❌ Could not load any images for PDF test")
        return
    
    print(f"🎯 Generating enhanced PDF report for: {category} waste")
    print(f"🖼️ Source image: {os.path.basename(image_path)}")
    
    # Process through pipeline
    print("🔍 Processing through EcoSight pipeline...")
    pipeline_result = await orchestrator.process_single_image(
        image_data=image_data,
        user_id=f"enhanced_pdf_test_{category}",
        location="NYC"
    )
    
    if pipeline_result["success"]:
        if 'pdf_bytes' in pipeline_result["report"]:
            report = pipeline_result["report"]
            pdf_bytes = report['pdf_bytes']
            pdf_filename = report['pdf_filename']
            
            # Display enhanced PDF information
            print("✅ Enhanced PDF Generation Successful!")
            PDFDownloader.display_pdf_info(pdf_bytes, pdf_filename)
            
            # Save to output directory
            saved_path = PDFDownloader.save_pdf_to_output(pdf_bytes, pdf_filename)
            
            print("\n" + "="*60)
            print("🎨 ENHANCED PDF PREVIEW OPTIONS:")
            print("="*60)
            
            # Option 1: Interactive Dashboard (Recommended)
            print("\n1. 🚀 INTERACTIVE DASHBOARD (Recommended):")
            dashboard = PDFDownloader.create_interactive_pdf_dashboard(pdf_bytes, pdf_filename)
            display(dashboard)
            
            # Option 2: Simple Preview
            print("\n2. 👀 SIMPLE PREVIEW:")
            simple_preview = PDFDownloader.create_simple_preview(pdf_bytes, pdf_filename)
            display(simple_preview)
            
            # Option 3: Standard Download Link
            print("\n3. 📥 STANDARD DOWNLOAD:")
            download_link = PDFDownloader.create_download_link(pdf_bytes, pdf_filename, "📥 Download Enhanced Report")
            display(download_link)
            
            # Report summary
            print("\n📋 ENHANCED REPORT SUMMARY:")
            print(f"   📊 Items Analyzed: {report['summary']['total_items']}")
            print(f"   ♻️  Recyclable: {report['summary']['recyclable_percent']:.1f}%")
            print(f"   🌱 Compost: {report['summary']['compost_percent']:.1f}%")
            print(f"   🗑️  Landfill: {report['summary']['landfill_percent']:.1f}%")
            print(f"   🌍 CO2 Saved: {report['environmental_impact']['co2_saved_kg']:.1f} kg")
            
            # Show preview tips
            print("\n💡 PREVIEW TIPS:")
            print("   • Use the Interactive Dashboard for best experience")
            print("   • Toggle preview to view PDF without downloading")
            print("   • Download multiple copies if needed")
            print("   • Check Kaggle output tab for saved file")
            
        else:
            print("❌ PDF generation failed - no PDF data in report")
    else:
        print("❌ Pipeline processing failed")
        print(f"   Error: {pipeline_result.get('error', 'Unknown error')}")

# Run enhanced PDF generation test
print("🚀 Starting enhanced PDF generation test...")
await test_enhanced_pdf_generation()

print("\n" + "="*60)
print("🎯 SINGLE COMPREHENSIVE PDF GENERATED SUCCESSFULLY!")
print("💡 Features demonstrated:")
print("   • Professional PDF generation with branding")
print("   • Interactive preview dashboard")
print("   • Multiple download options")
print("   • Environmental impact analytics")
print("   • Waste classification results")

📄 Testing Enhanced PDF Report Generation with Preview...
🚀 Starting enhanced PDF generation test...
🔄 Loading waste image for enhanced PDF test...
🎯 Generating enhanced PDF report for: plastic waste
🖼️ Source image: plastic460.jpg
🔍 Processing through EcoSight pipeline...
🔍 Starting Vision Analysis...
🎯 Attempting Gemini Vision analysis...
🔮 Calling real Gemini Vision API with model: models/gemini-2.5-flash...
❌ Gemini Vision API error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10, model: gemini-2.5-flash
Please retry in 43.57962787s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  


2. 👀 SIMPLE PREVIEW:



3. 📥 STANDARD DOWNLOAD:



📋 ENHANCED REPORT SUMMARY:
   📊 Items Analyzed: 2
   ♻️  Recyclable: 100.0%
   🌱 Compost: 0.0%
   🗑️  Landfill: 0.0%
   🌍 CO2 Saved: 1.0 kg

💡 PREVIEW TIPS:
   • Use the Interactive Dashboard for best experience
   • Toggle preview to view PDF without downloading
   • Download multiple copies if needed
   • Check Kaggle output tab for saved file

🎯 SINGLE COMPREHENSIVE PDF GENERATED SUCCESSFULLY!
💡 Features demonstrated:
   • Professional PDF generation with branding
   • Interactive preview dashboard
   • Multiple download options
   • Environmental impact analytics
   • Waste classification results
